In [2]:
import pandas as pd
import numpy as np
import pandas_profiling
#import seaborn as sns
import wget
import tabula
from datetime import date, timedelta
import datetime
from pathlib import Path

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
#pio.templates.default = "plotly_dark"
from plotly.subplots import make_subplots

pd.set_option('display.max_columns',99)
pd.set_option('display.max_rows',200)

today = date.today()

In [7]:
#submission = pd.read_csv('submission.csv')
#train = pd.read_csv('train.csv')

In [3]:
datedf = '3/17/20'
datedf = datetime.datetime.strptime(datedf, '%m/%d/%y').date()
print(datedf)

2020-03-17


#### Update Date Ranges

In [3]:
#function to return days inbetween a specified range. To be used to pic up URLS for the new NYC
#historical data on the DOH website.

def daterange(date1, date2):
    #l=[]
    for n_days in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n_days)
   

start_dt = date(2020, 3, 17)
end_dt = date(2020, 4, 6) # <------update date!!!!!
for dt in daterange(start_dt, end_dt):
    print(dt.strftime("%Y-%m-%d"))

2020-03-17
2020-03-18
2020-03-19
2020-03-20
2020-03-21
2020-03-22
2020-03-23
2020-03-24
2020-03-25
2020-03-26
2020-03-27
2020-03-28
2020-03-29
2020-03-30
2020-03-31
2020-04-01
2020-04-02
2020-04-03
2020-04-04
2020-04-05
2020-04-06


In [4]:
datescases = [dt.strftime("%m%d%Y") for dt in daterange(date(2020,3,17),date(2020,4,6))]
datesdeaths = [dt.strftime("%m%d%Y") for dt in daterange(date(2020,3,22),date(2020,4,6))]
dateshosp = [dt.strftime("%m%d%Y") for dt in daterange(date(2020,3,24),date(2020,4,6))]

In [5]:
datescases = [dates+'-2' for dates in datescases]
datescases

['03172020-2',
 '03182020-2',
 '03192020-2',
 '03202020-2',
 '03212020-2',
 '03222020-2',
 '03232020-2',
 '03242020-2',
 '03252020-2',
 '03262020-2',
 '03272020-2',
 '03282020-2',
 '03292020-2',
 '03302020-2',
 '03312020-2',
 '04012020-2',
 '04022020-2',
 '04032020-2',
 '04042020-2',
 '04052020-2',
 '04062020-2']

In [6]:
datesdeaths = [dates+'-2' for dates in datesdeaths]
datesdeaths

['03222020-2',
 '03232020-2',
 '03242020-2',
 '03252020-2',
 '03262020-2',
 '03272020-2',
 '03282020-2',
 '03292020-2',
 '03302020-2',
 '03312020-2',
 '04012020-2',
 '04022020-2',
 '04032020-2',
 '04042020-2',
 '04052020-2',
 '04062020-2']

In [7]:
dateshosp = [dates+'-1' for dates in dateshosp]
dateshosp

['03242020-1',
 '03252020-1',
 '03262020-1',
 '03272020-1',
 '03282020-1',
 '03292020-1',
 '03302020-1',
 '03312020-1',
 '04012020-1',
 '04022020-1',
 '04032020-1',
 '04042020-1',
 '04052020-1',
 '04062020-1']

### **I. NYC Specific Data**

In [8]:
folderpath = Path('/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc')
# file_to_open = folderpath / 'covid-19-daily-data-summary-hospitalizations.pdf'
# files = [folderpath / 'covid-19-daily-data-summary-hospitalizations.pdf',folderpath / 'covid-19-daily-data-summary-deaths.pdf',folderpath / 'covid-19-daily-data-summary.pdf']

# for i in range(len(files)):
    
#     tabula.convert_into(files[i],names[i], output_format="csv", pages='all')

In [10]:
pdfdate=datedf.strftime("%m%d%Y")
pdfdate = pdfdate+'-2.pdf'
pdfdate

'03172020-2.pdf'

In [9]:
dates = [dateshosp,datesdeaths,datescases]
dates[0]

['03242020-1',
 '03252020-1',
 '03262020-1',
 '03272020-1',
 '03282020-1',
 '03292020-1',
 '03302020-1',
 '03312020-1',
 '04012020-1',
 '04022020-1',
 '04032020-1',
 '04042020-1',
 '04052020-1',
 '04062020-1']

In [15]:
latestdate = [dates[0][-1],dates[1][-1],dates[2][-1]]
latestdate

['04062020-1', '04062020-2', '04062020-2']

In [26]:
# convert PDFs into CSV, read CSVs into DataFrames.

urls = ['https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary-hospitalizations-',
        "https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary-deaths-",
        'https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary-']

names = ['/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc/NYCH',
         '/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc/NYCD',
         '/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc/NYCC']

#===============================================================
#===============================================================
#-----------FOR CONSTRUCTING NEW ITD FILES ONLY!!!!!!! COMMENT OUT if updating 'latestdate'
newurls = [urls[i]+dates[i][j]+'.pdf' for i in range(len(urls)) for j in range(len(dates[i]))]
newnames = [names[i]+dates[i][j]+'.csv' for i in range(len(names)) for j in range(len(dates[i]))]

# for i in range(len(newurls)):
    
#     tabula.convert_into(newurls[i],newnames[i], output_format="csv", pages='all')
#===============================================================
#===============================================================


##                reading in CSVs for LATEST DATE ONLY! 
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


latesturls = [urls[i]+latestdate[i]+'.pdf' for i in range(len(urls))]
latestnames = [names[i]+latestdate[i]+'.csv' for i in range(len(names))]

for i in range(len(latesturls)):
    
     tabula.convert_into(latesturls[i],latestnames[i], output_format="csv", pages='all')


#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

files = ['NYCD','NYCH','NYCC']
#select file below using by indexing into the files list:
positionsd = [idx for idx, s in enumerate(newnames) if files[0] in s]
positionsh = [idx for idx, s in enumerate(newnames) if files[1] in s]
positionsc = [idx for idx, s in enumerate(newnames) if files[2] in s]
startdeath = positionsd[0]
enddeath = positionsd[-1]
starthosp = positionsh[0]
endhosp = positionsh[-1]
startcases = positionsc[0]
endcases = positionsc[-1]

#===============================================================
#ORIGINAL CODE BELOW (to update 1 date at a time)

# for i in range(len(newurls)):
    
#     tabula.convert_into(urls[i],names[i], output_format="csv", pages='all')
    
# nycd = pd.read_csv(folderpath / 'NYCD.csv', header = None)
# nych = pd.read_csv(folderpath / 'NYCH.csv', header = None)
# nycc = pd.read_csv(folderpath / 'NYCC.csv')

In [27]:
print('Deaths positions:',startdeath,enddeath,'\nHosp positions:',starthosp,endhosp,
      '\nCases positions:',startcases,endcases)

Deaths positions: 14 29 
Hosp positions: 0 13 
Cases positions: 30 50


In [690]:
#newnames

==========**Below Dataframes only for ITD using new urls and new names from above. Select files in previous cell FIRST before running the dataframes below**==========

In [38]:
nycddf = [pd.read_csv(newnames[startdeath+i], header = None) for i in range(len(newnames[startdeath:enddeath+1]))]

In [39]:
nychdf = [pd.read_csv(newnames[starthosp+i], header = None) for i in range(len(newnames[starthosp:endhosp+1]))]

In [40]:
nyccdf = [pd.read_csv(newnames[startcases+i], header = None) for i in range(len(newnames[startcases:endcases+1]))]

In [31]:
# i=2
# pd.read_csv(newnames[startcases+i], header = None)

In [193]:
#nyccfiles = newnames[startidx:endidx+1]

In [32]:
#[dateshosp,datesdeaths,datescases]
dates_dict = {'dateshosp':dates[0],'datesdeaths':dates[1],'datescases':dates[2]}
dates_dict

{'dateshosp': ['03242020-1',
  '03252020-1',
  '03262020-1',
  '03272020-1',
  '03282020-1',
  '03292020-1',
  '03302020-1',
  '03312020-1',
  '04012020-1',
  '04022020-1',
  '04032020-1',
  '04042020-1',
  '04052020-1',
  '04062020-1'],
 'datesdeaths': ['03222020-2',
  '03232020-2',
  '03242020-2',
  '03252020-2',
  '03262020-2',
  '03272020-2',
  '03282020-2',
  '03292020-2',
  '03302020-2',
  '03312020-2',
  '04012020-2',
  '04022020-2',
  '04032020-2',
  '04042020-2',
  '04052020-2',
  '04062020-2'],
 'datescases': ['03172020-2',
  '03182020-2',
  '03192020-2',
  '03202020-2',
  '03212020-2',
  '03222020-2',
  '03232020-2',
  '03242020-2',
  '03252020-2',
  '03262020-2',
  '03272020-2',
  '03282020-2',
  '03292020-2',
  '03302020-2',
  '03312020-2',
  '04012020-2',
  '04022020-2',
  '04032020-2',
  '04042020-2',
  '04052020-2',
  '04062020-2']}

In [33]:
def prepdfsinlist(list_of_dfs,dict_key):
    '''
    Given a list of dataframes and a dictionary of dates, for each dataframe in the list
    eliminate all columns that only have NaNs down all rows within that column.
    Additionally, for each dataframe in the list, append the corresponding date as a 'Date' column,
    so that every dataframe can be identified by the date that the data corresponds to.
    '''
    #drop columns with all NaNs in all the dataframes in the list.
    list_of_dfs = [df.dropna(axis = 1,how = 'all') for df in list_of_dfs]
  
    #for each dataframe, standardize column names to index numbers and append a 'Date' column
    for i in range(len(dict_key)):
        columns = list(range(list_of_dfs[i].shape[1]))
        list_of_dfs[i].columns = columns
        temp_date = dict_key[i][:len(dict_key[i])-2]
        temp_date = datetime.datetime.strptime(temp_date, '%m%d%Y').date()
        list_of_dfs[i].loc[:,'Date'] = temp_date
    return list_of_dfs

In [34]:
#This cell is used to index into the list of dataframes. This is used to check the prepdfsinlist
#function is working properly and that all dfs have dates appended.

selector = 5
selected_date_c = dates_dict['datescases'][selector][:len(dates_dict['datescases'][selector])-2]
selected_date_c = datetime.datetime.strptime(selected_date_c, '%m%d%Y').date()
selected_date_h = dates_dict['dateshosp'][selector][:len(dates_dict['dateshosp'][selector])-2]
selected_date_h = datetime.datetime.strptime(selected_date_h, '%m%d%Y').date()
selected_date_d = dates_dict['datesdeaths'][selector][:len(dates_dict['datesdeaths'][selector])-2]
selected_date_d = datetime.datetime.strptime(selected_date_d, '%m%d%Y').date()


print('Cases selected date:',selected_date_c)
print('Hosp selected date:',selected_date_h)
print('Deaths selected date:',selected_date_d)

Cases selected date: 2020-03-22
Hosp selected date: 2020-03-29
Deaths selected date: 2020-03-27


In [41]:
nyccdf = prepdfsinlist(nyccdf,dates_dict['datescases'])
nychdf = prepdfsinlist(nychdf,dates_dict['dateshosp'])
nycddf = prepdfsinlist(nycddf,dates_dict['datesdeaths'])

In [37]:
del nyccdf
del nychdf
del nycddf

NameError: name 'nyccdf' is not defined

In [42]:
dates_dict['dateshosp']

['03242020-1',
 '03252020-1',
 '03262020-1',
 '03272020-1',
 '03282020-1',
 '03292020-1',
 '03302020-1',
 '03312020-1',
 '04012020-1',
 '04022020-1',
 '04032020-1',
 '04042020-1',
 '04052020-1',
 '04062020-1']

In [45]:
dcats = ['Age Group','Sex','Borough','Total']
hcats = ['Age Group','Sex','Borough','Total']
ccats = ['Age Group','Age 50 and over','Sex','Borough','Deaths']

### DataFrame Construction Functions

In [52]:
def getalldfs(dfs,categories,dict_key):
    
    '''Given a list of dataframes of the desired metric (deaths, hospitalizations, or cases), 
       a list of categories specific to that metric, and the dictionary key specifying the
       dates on which the metric was measured, return time series dataframes for each of the
       specified metrics, split out by category. 
    '''
    
    #outer loop iterates through the passed in categories that pertain to the passed in list of
    #dataframes.
    
    for i in range(len(categories)-1):
        temp_df = pd.DataFrame()    
        
        #this inner loop cycles through all the dates that are "attached" to the passed in list of
        #dataframes. 
        
        for e in range(len(dict_key)):
            #these assignments tell the function to pick up only those indices that cover the
            #current category
            
            #Hospitalizations+++++++++
            if dfs == nychdf: #ok   
                temp = dfs[e][getIndices(dfs[e],categories[i])[0][0]:getIndices(dfs[e],
                       categories[i+1])[0][0]].rename({0:categories[i],
                                                       1:'Ever Hospitalized Cases',
                                                       2:'Total Cases'}, axis = 1).reset_index(drop=True)
                         
            #Deaths+++++++++++++++++++    
            elif dfs == nycddf: #for nyccdf, requires taking multpile cases into account.
                temp = dfs[e][getIndices(dfs[e],categories[i])[0][0]:getIndices(dfs[e],
                                                                                      categories[i+1])[0][0]].reset_index(drop=True)
                if e < 5 and i == 2:
                    temp = temp[0:6]
                if e < 5:
                    #temp = temp.drop(0,axis = 0).reset_index(drop=True)
                    #temp = temp.drop(0, axis = 0)
                    temp = temp.rename({0:categories[i],1:'Total Deaths'}, axis = 1)
                    temp.loc[:,'Underlying Conditions'] = 0
                    temp.loc[:,'No Underlying Conditions'] = 0
                    temp.loc[:,'Underlying Conditions Pending'] = 0
                    temp = temp[[categories[i],'Underlying Conditions','No Underlying Conditions',
                                'Underlying Conditions Pending','Total Deaths','Date']]
                    
                else:
                    temp=temp.rename({0:categories[i],1:'Underlying Conditions',
                                      2:'No Underlying Conditions',
                                      3:'Underlying Conditions Pending',
                                      4:'Total Deaths'}, axis=1)
            
            #Cases++++++++++++++++++++
            elif dfs == nyccdf: #again, for nyccdf, various cases need to be taken into account.
                if i==1 and e==0:
                    temp = pd.DataFrame({categories[i]:['Blank','- Yes','- No'],
                                         'Total Cases':[0,0,0]})
    
                    #temp = temp.rename({0:})
                    temp_date = dates_dict['datescases'][e][:-2]
                    temp_date = datetime.datetime.strptime(temp_date, '%m%d%Y').date()
                    temp.loc[:,'Date'] = temp_date

                else:
                    try:
                        temp = dfs[e][getIndices(dfs[e],categories[i])[0][0]:getIndices(dfs[e],
                               categories[i+1])[0][0]].reset_index(drop = True).rename({0:categories[i],
                                                                                        1:'Total Cases'},
                                                                        axis = 1).reset_index(drop=True)
                    except:
                        temp = dfs[e][getIndices(dfs[e],categories[i])[0][0]:getIndices(dfs[e],
                               'Sex')[0][0]].rename({0:categories[i],1:'Total Cases'},axis = 1).reset_index(drop=True)
        
            #format by dropping uneeded rows, then finally concatenate all the mini-dataframes
            #into one big dataframe with all of the desired dates, creating time series data.
            
            temp = temp.drop(0, axis = 0).reset_index(drop = True)

            temp[categories[i]] = temp[categories[i]].map(lambda x: x[1:].lstrip())
            #use applymap (instead of apply) for element-wise usage of the countform function.
            temp.iloc[:,1:-1] = temp.iloc[:,1:-1].applymap(countform)#.astype(int)
            #temp['Date'] = today #today
            temp_df = pd.concat([temp_df,temp], axis = 0, sort = False).reset_index(drop = True)

           
        yield temp_df

In [48]:
def getIndices(df, value):
    '''Given a dataframe and a value, returns the index of the value
       within the dataframe, otherwise returns -1.
    '''
    #get bool dataframe showing whether the object is actually in the dataframe or not.
    booldf = df.isin([value])
    #show which columns have the object with bool True
    boolseries = booldf.any()
    #shrink the result so that only True shows up. Store the successful columns in a list.
    colnames = list(boolseries[boolseries == True].index)
    #iterate through the columns to check
    
    results = []
    
    for col in colnames:
        #store all matching rows into a list
        rows = list(booldf[col][booldf[col] == True].index)
        for row in rows:
            results.append((row,col)) #append tuple of row, column to result list.
    
    return results
    

#format the value column.
def countform(rowitem):
    if str(rowitem).find('(') != -1:
        return int(rowitem[:rowitem.find('(')-1])
    else:
        return int(rowitem) #int(rowitem[:rowitem.find('(')-1])

In [872]:
#del temp

In [49]:
i = 0
e = 3
categories = hcats
temp = nychdf[e][getIndices(nychdf[e],categories[i])[0][0]:getIndices(nychdf[e],categories[i+1])[0][0]].rename({0:categories[i], 1:'Ever Hospitalized Cases', 2:'Total Cases'}, axis = 1)
temp.drop(0, axis = 0, inplace = True)

temp[categories[i]] = temp[categories[i]].map(lambda x: x[1:].lstrip())
temp.iloc[:,1] = temp.iloc[:,1].apply(countform)
#temp['Date'] = today #today
#temp_df = pd.concat([temp_df,temp], axis = 0, sort = False).reset_index(drop = True)
    
temp
#temp_df = pd.concat([temp_df,temp], axis = 0, sort = False).reset_index(drop = True)
#temp_df

Age Group  Ever Hospitalized Cases  Total Cases        Date
1      0 to 17                       47        543.0  2020-03-27
2     18 to 44                      971      11617.0  2020-03-27
3     45 to 64                     1886       9158.0  2020-03-27
4     65 to 74                     1032       3034.0  2020-03-27
5  75 and over                     1103       2286.0  2020-03-27
6      Unknown                        0         59.0  2020-03-27

In [ ]:
ccats = ['Age Group','Age 50 and over','Sex','Borough','Deaths']

In [50]:
i = 3
e = 15
categories = ccats

if i==1 and e==0:
    temp = pd.DataFrame({categories[i]:['Blank','- Yes','- No'],'Total Cases':[0,0,0]})
    #temp = temp.rename({0:})
    temp_date = dates_dict['datescases'][e][:-2]
    temp_date = datetime.datetime.strptime(temp_date, '%m%d%Y').date()
    temp.loc[:,'Date'] = temp_date

else:
    try:
        temp = nyccdf[e][getIndices(nyccdf[e],categories[i])[0][0]:getIndices(nyccdf[e],
               categories[i+1])[0][0]].reset_index(drop = True).rename({0:categories[i],
                                                                        1:'Total Cases'},
                                                                        axis = 1).reset_index(drop=True)
    except:
        temp = nyccdf[e][getIndices(nyccdf[e],categories[i])[0][0]:getIndices(nyccdf[e],
               'Sex')[0][0]].rename({0:categories[i],1:'Total Cases'},axis = 1).reset_index(drop=True)
    

    
temp = temp.drop(0, axis = 0).reset_index(drop=True)

temp[categories[i]] = temp[categories[i]].map(lambda x: x[1:].lstrip())
temp.iloc[:,1:-1] = temp.iloc[:,1:-1].applymap(countform)

#temp_df = pd.concat([temp_df,temp], axis = 0, sort = False).reset_index(drop = True)
    
temp
#temp_df = pd.concat([temp_df,temp], axis = 0, sort = False).reset_index(drop = True)
#temp_df

Borough Total Cases        Date
0          Bronx        8607  2020-04-01
1       Brooklyn       12274  2020-04-01
2      Manhattan        7022  2020-04-01
3         Queens       15217  2020-04-01
4  Staten Island        2552  2020-04-01
5        Unknown          35  2020-04-01

In [406]:
if temp.iloc[1,1].find('(') != -1:
    print('Found')
else:
    print('Not Found')

Not Found


In [454]:
i = 2
e = 3 #dataframe index 2 needs to be fixed
categories = dcats
temp = nycddf[e][getIndices(nycddf[e],categories[i])[0][0]:getIndices(nycddf[e],categories[i+1])[0][0]].reset_index(drop=True)#.rename({0:categories[i], 1:'Ever Hospitalized Cases', 2:'Total Cases'}, axis = 1)
if e < 5 and i == 2:
    temp = temp[0:6]
#if temp.shape[1]<4
if e < 5:
    #temp = temp.drop(0,axis = 0).reset_index(drop=True)
    #temp = temp.drop(0, axis = 0)
    temp = temp.rename({0:categories[i],1:'Total Deaths'}, axis = 1)
    #temp.iloc[:,1] = temp.iloc[:,1].apply(countform)
    temp.loc[:,'Underlying Conditions'] = 0
    temp.loc[:,'No Underlying Conditions'] = 0
    temp.loc[:,'Underlying Conditions Pending'] = 0
    temp = temp[[categories[i],'Underlying Conditions','No Underlying Conditions',
                 'Underlying Conditions Pending','Total Deaths','Date']]
    
    
else:
    temp=temp.rename({0:categories[i],1:'Underlying Conditions',2:'No Underlying Conditions',
                      3:'Underlying Conditions Pending',4:'Total Deaths'}, axis=1)
    
temp = temp.drop(0, axis = 0).reset_index(drop=True)
temp[categories[i]] = temp[categories[i]].map(lambda x: x[1:].lstrip())
temp.iloc[:,-2] = temp.iloc[:,-2].apply(countform)

temp
#temp_df = pd.concat([temp_df,temp], axis = 0, sort = False).reset_index(drop = True)
#temp_df

Borough  Underlying Conditions  No Underlying Conditions  \
0          Bronx                      0                         0   
1       Brooklyn                      0                         0   
2      Manhattan                      0                         0   
3         Queens                      0                         0   
4  Staten Island                      0                         0   

   Underlying Conditions Pending  Total Deaths        Date  
0                              0            60  2020-03-25  
1                              0            64  2020-03-25  
2                              0            43  2020-03-25  
3                              0            92  2020-03-25  
4                              0            21  2020-03-25

In [496]:
nychdf[1]

0           1        2        Date
0          Age Group         NaN      NaN  2020-03-25
1         -  0 to 17     35 (8%)    446.0  2020-03-25
2        -  18 to 44    837 (9%)   8880.0  2020-03-25
3        -  45 to 64  1470 (22%)   6786.0  2020-03-25
4        -  65 to 74   736 (33%)   2226.0  2020-03-25
5     -  75 and over   844 (52%)   1633.0  2020-03-25
6         -  Unknown      0 (0%)     40.0  2020-03-25
7                Sex         NaN      NaN  2020-03-25
8          -  Female  1628 (19%)   8655.0  2020-03-25
9            -  Male  2293 (20%)  11325.0  2020-03-25
10        -  Unknown      1 (3%)     31.0  2020-03-25
11           Borough         NaN      NaN  2020-03-25
12          -  Bronx   802 (23%)   3542.0  2020-03-25
13       -  Brooklyn   919 (18%)   5232.0  2020-03-25
14      -  Manhattan   584 (16%)   3616.0  2020-03-25
15         -  Queens  1363 (21%)   6420.0  2020-03-25
16  -  Staten Island   251 (22%)   1166.0  2020-03-25
17        -  Unknown      3 (9%)     35.0  2020-03-25
18             Total  3922 (20%)  20011.0  2020-03-25

### Create DataFrames

====================**Create DataFrames Here**=================

In [51]:
age_hosp,sex_hosp,borough_hosp = getalldfs(nychdf,hcats,dates_dict['dateshosp'])
age_deaths,sex_deaths,borough_deaths = getalldfs(nycddf,dcats,dates_dict['datesdeaths'])
age_cases,cases50,sex_cases,borough_cases = getalldfs(nyccdf,ccats,dates_dict['datescases'])

=========================================================

Confirm integrity and fix issues

In [100]:
#function to repair bad values.
def fixvalues(serieselement):
    try:
        idx = correction['bad'].index(serieselement)
        serieselement = correction['good'][idx]
    except:
        serieselement = serieselement
    return serieselement

In [89]:
age_hosp

Age Group  Ever Hospitalized Cases  Total Cases        Date
0       0 to 17                       28          384  2020-03-24
1      18 to 44                      629         7094  2020-03-24
2      45 to 64                     1061         5194  2020-03-24
3      65 to 74                      550         1689  2020-03-24
4   75 and over                      615         1227  2020-03-24
5       Unknown                        0            9  2020-03-24
6       0 to 17                       35          446  2020-03-25
7      18 to 44                      837         8880  2020-03-25
8      45 to 64                     1470         6786  2020-03-25
9      65 to 74                      736         2226  2020-03-25
10  75 and over                      844         1633  2020-03-25
11      Unknown                        0           40  2020-03-25
12      0 to 17                       46          495  2020-03-26
13     18 to 44                      950        10145  2020-03-26
14     45 to 64                     1749         7869  2020-03-26
15     65 to 74                      946         2627  2020-03-26
16  75 and over                     1029         1935  2020-03-26
17      Unknown                        0           41  2020-03-26
18      0 to 17                       47          543  2020-03-27
19     18 to 44                      971        11617  2020-03-27
20     45 to 64                     1886         9158  2020-03-27
21     65 to 74                     1032         3034  2020-03-27
22  75 and over                     1103         2286  2020-03-27
23      Unknown                        0           59  2020-03-27
24      0 to 17                       58          591  2020-03-28
25     18 to 44                     1224        13213  2020-03-28
26     45 to 64                     2350        10596  2020-03-28
27     65 to 74                     1267         3571  2020-03-28
28  75 and over                     1388         2724  2020-03-28
29      Unknown                        0           70  2020-03-28
30      0 to 17                       67          619  2020-03-29
31     18 to 44                     1459        14233  2020-03-29
32     45 to 64                     2765        11577  2020-03-29
33     65 to 74                     1499         3954  2020-03-29
34  75 and over                     1620         3020  2020-03-29
35      Unknown                        0           71  2020-03-29
36      0 to 17                       72          714  2020-03-30
37     18 to 44                     1448        16028  2020-03-30
38     45 to 64                     2887        13344  2020-03-30
39     65 to 74                     1612         4496  2020-03-30
40  75 and over                     1722         3410  2020-03-30
41      Unknown                        0           95  2020-03-30
42      0 to 17                       74          757  2020-03-31
43     18 to 44                     1532        17347  2020-03-31
44     45 to 64                     3195        14689  2020-03-31
45     65 to 74                     1826         5015  2020-03-31
46  75 and over                     1922         3866  2020-03-31
47      Unknown                        0           97  2020-03-31
48      0 to 17                       83          816  2020-04-01
49     18 to 44                     1711        18767  2020-04-01
50     45 to 64                     3658        16104  2020-04-01
51     65 to 74                     2114         5584  2020-04-01
52  75 and over                     2209         4328  2020-04-01
53      Unknown                        0          108  2020-04-01
54      0 to 17                       88          881  2020-04-02
55     18 to 44                     1817        20313  2020-04-02
56     45 to 64                     3962        17497  2020-04-02
57     65 to 74                     2300         6095  2020-04-02
58  75 and over                     2423         4811  2020-04-02
59      Unknown                        0      

In [103]:
age_deaths

Age Group  Underlying Conditions  No Underlying Conditions  \
0       0 to 17                      0                         0   
1      18 to 44                      0                         0   
2      45 to 64                      0                         0   
3      65 to 74                      0                         0   
4   75 and over                      0                         0   
5       0 to 17                      0                         0   
6      18 to 44                      0                         0   
7      45 to 64                      0                         0   
8      65 to 74                      0                         0   
9   75 and over                      0                         0   
10      0 to 17                      0                         0   
11     18 to 44                      0                         0   
12     45 to 64                      0                         0   
13     65 to 74                      0                         0   
14  75 and over                      0                         0   
15      0 to 17                      0                         0   
16     18 to 44                      0                         0   
17     45 to 64                      0                         0   
18     65 to 74                      0                         0   
19  75 and over                      0                         0   
20      0 to 17                      0                         0   
21     18 to 44                      0                         0   
22     45 to 64                      0                         0   
23     65 to 74                      0                         0   
24  75 and over                      0                         0   
25      0 to 17                      0                         0   
26     18 to 44                     15                         1   
27     45 to 64                     79                         7   
28     65 to 74                     90                         2   
29  75 and over                    189                         4   
30      0 to 17                      0                         0   
31     18 to 44                     23                         1   
32     45 to 64                    125                         8   
33     65 to 74                    125                         2   
34  75 and over                    254                         6   
35      0 to 17                      1                         0   
36     18 to 44                     28                         1   
37     45 to 64                    140                         8   
38     65 to 74                    152                         1   
39  75 and over                    303                         3   
40      0 to 17                      1                         0   
41     18 to 44                     39                         2   
42     45 to 64                    170                         9   
43     65 to 74                    167                         0   
44  75 and over                    327                         3   
45      0 to 17                      1                         0   
46     18 to 44                     45                         2   
47     45 to 64                    187                         9   
48     65 to 74                    187                         0   
49  75 and over                    351                         3   
50      0 to 17                      1                         0   
51     18 to 44                     51                         4   
52     45 to 64                    254                         9   
53     65 to 74                    257                         1   
54  75 and over                    431                         4   
55      0 to 17                      1                         0   
56     18 to 44                     64                         7   
57     45 to 64                    304                        

In [102]:
age_deaths.loc[:,'Age Group'] = age_deaths.loc[:,'Age Group'].map(fixvalues)

In [108]:
age_cases

Age Group Total Cases        Date
0         0 to 4           0  2020-03-17
1        5 to 17          25  2020-03-17
2       18 to 49         457  2020-03-17
3       50 to 64         211  2020-03-17
4    65 and over         230  2020-03-17
5         0 to 4           5  2020-03-18
6        5 to 17          42  2020-03-18
7       18 to 49        1034  2020-03-18
8       50 to 64         463  2020-03-18
9    65 and over         464  2020-03-18
10       Unknown           1  2020-03-18
11        0 to 4          14  2020-03-19
12       5 to 17          94  2020-03-19
13      18 to 49        2117  2020-03-19
14      50 to 64         926  2020-03-19
15   65 and over         800  2020-03-19
16       Unknown           3  2020-03-19
17        0 to 4          26  2020-03-20
18       5 to 17         131  2020-03-20
19      18 to 49        3080  2020-03-20
20      50 to 64        1330  2020-03-20
21   65 and over        1112  2020-03-20
22       Unknown           4  2020-03-20
23        0 to 4          34  2020-03-21
24       5 to 17         189  2020-03-21
25      18 to 49        4456  2020-03-21
26      50 to 64        1929  2020-03-21
27   65 and over        1502  2020-03-21
28       Unknown           5  2020-03-21
29       0 to 17         300  2020-03-22
30      18 to 44        4965  2020-03-22
31      45 to 64        3519  2020-03-22
32      65 to 74        1150  2020-03-22
33   75 and over         825  2020-03-22
34       Unknown           5  2020-03-22
35       0 to 17         341  2020-03-23
36      18 to 44        6035  2020-03-23
37      45 to 64        4343  2020-03-23
38      65 to 74        1402  2020-03-23
39   75 and over         988  2020-03-23
40       Unknown          10  2020-03-23
41       0 to 17         384  2020-03-24
42      18 to 44        7094  2020-03-24
43      45 to 64        5194  2020-03-24
44      65 to 74        1689  2020-03-24
45   75 and over        1227  2020-03-24
46       Unknown           9  2020-03-24
47       0 to 17         446  2020-03-25
48      18 to 44        8880  2020-03-25
49      45 to 64        6786  2020-03-25
50      65 to 74        2226  2020-03-25
51   75 and over        1633  2020-03-25
52       Unknown          40  2020-03-25
53       0 to 17         495  2020-03-26
54      18 to 44       10145  2020-03-26
55      45 to 64        7869  2020-03-26
56      65 to 74        2627  2020-03-26
57   75 and over        1935  2020-03-26
58       Unknown          41  2020-03-26
59       0 to 17         543  2020-03-27
60      18 to 44       11617  2020-03-27
61      45 to 64        9158  2020-03-27
62      65 to 74        3034  2020-03-27
63   75 and over        2286  2020-03-27
64       Unknown          59  2020-03-27
65       0 to 17         591  2020-03-28
66      18 to 44       13213  2020-03-28
67      45 to 64       10596  2020-03-28
68      65 to 74        3571  2020-03-28
69   75 and over        2724  2020-03-28
70       Unknown          70  2020-03-28
71       0 to 17         619  2020-03-29
72      18 to 44       14233  2020-03-29
73      45 to 64       11577  2020-03-29
74      65 to 74        3954  2020-03-29
75   75 and over        3020  2020-03-29
76       Unknown          71  2020-03-29
77       0 to 17         714  2020-03-30
78      18 to 44       16028  2020-03-30
79      45 to 64       13344  2020-03-30
80      65 to 74        4496  2020-03-30
81   75 and over        3410  2020-03-30
82       Unknown          95  2020-03-30
83       0 to 17         757  2020-03-31
84      18 to 44       17347  2020-03-31
85      45 to 64       14689  2020-03-31
86      65 to 74        5015  2020-03-31
87   75 and over        3866  2020-03-31
88       Unknown          97  2020-03-31
89       0 to 17         816  2020-04-01
90      18 to 44       18767  2020-04-01
91      45 to 64       16104  2020-04-01
92      65 to 74        5584  2020-04-01
93   75 and over        4328  2020-04-01
94       Unknown         108  2020-04-01
95       0 to 17         881  2020-04-02
96      18 to 44       20313  

In [107]:
age_cases.loc[:,'Age Group'] = age_cases.loc[:,'Age Group'].map(fixvalues)

In [109]:
sex_hosp

Sex  Ever Hospitalized Cases  Total Cases        Date
0    Female                     1191         6736  2020-03-24
1      Male                     1691         8838  2020-03-24
2   Unknown                        1           23  2020-03-24
3    Female                     1628         8655  2020-03-25
4      Male                     2293        11325  2020-03-25
5   Unknown                        1           31  2020-03-25
6    Female                     1918        10124  2020-03-26
7      Male                     2801        12948  2020-03-26
8   Unknown                        1           40  2020-03-26
9    Female                     2035        11792  2020-03-27
10     Male                     3003        14863  2020-03-27
11  Unknown                        1           42  2020-03-27
12   Female                     2545        13592  2020-03-28
13     Male                     3741        17133  2020-03-28
14  Unknown                        1           40  2020-03-28
15   Female                     3001        14837  2020-03-29
16     Male                     4408        18593  2020-03-29
17  Unknown                        1           44  2020-03-29
18   Female                     3130        16920  2020-03-30
19     Male                     4610        21120  2020-03-30
20  Unknown                        1           47  2020-03-30
21   Female                     3455        18677  2020-03-31
22     Male                     5093        23043  2020-03-31
23  Unknown                        1           51  2020-03-31
24   Female                     3980        20522  2020-04-01
25     Male                     5792        25128  2020-04-01
26  Unknown                        3           57  2020-04-01
27   Female                     4292        22485  2020-04-02
28     Male                     6295        27155  2020-04-02
29  Unknown                        3           67  2020-04-02
30   Female                     4765        25536  2020-04-03
31     Male                     6970        30672  2020-04-03
32  Unknown                        4           81  2020-04-03
33   Female                     5116        27660  2020-04-04
34     Male                     7596        33100  2020-04-04
35  Unknown                        4           90  2020-04-04
36   Female                     5714        29607  2020-04-05
37     Male                     8487        35256  2020-04-05
38  Unknown                        4           92  2020-04-05
39   Female                     6166        31319  2020-04-06
40     Male                     9163        37361  2020-04-06
41  Unknown                        4           96  2020-04-06

In [114]:
sex_cases

Sex Total Cases        Date
0    Female         378  2020-03-17
1      Male         536  2020-03-17
2    Female         850  2020-03-18
3      Male        1155  2020-03-18
4   Unknown           4  2020-03-18
5    Female        1630  2020-03-19
6      Male        2315  2020-03-19
7   Unknown           9  2020-03-19
8    Female        2317  2020-03-20
9      Male        3355  2020-03-20
10  Unknown          11  2020-03-20
11   Female        3318  2020-03-21
12     Male        4787  2020-03-21
13  Unknown          10  2020-03-21
14   Female        4522  2020-03-22
15     Male        6227  2020-03-22
16  Unknown          15  2020-03-22
17   Female        5612  2020-03-23
18     Male        7490  2020-03-23
19  Unknown          17  2020-03-23
20   Female        6736  2020-03-24
21     Male        8838  2020-03-24
22  Unknown          23  2020-03-24
23   Female        8655  2020-03-25
24     Male       11325  2020-03-25
25  Unknown          31  2020-03-25
26   Female       10124  2020-03-26
27     Male       12948  2020-03-26
28  Unknown          40  2020-03-26
29   Female       11792  2020-03-27
30     Male       14863  2020-03-27
31  Unknown          42  2020-03-27
32   Female       13592  2020-03-28
33     Male       17133  2020-03-28
34  Unknown          40  2020-03-28
35   Female       14837  2020-03-29
36     Male       18593  2020-03-29
37  Unknown          44  2020-03-29
38   Female       16920  2020-03-30
39     Male       21120  2020-03-30
40  Unknown          47  2020-03-30
41   Female       18677  2020-03-31
42     Male       23043  2020-03-31
43  Unknown          51  2020-03-31
44   Female       20522  2020-04-01
45     Male       25128  2020-04-01
46  Unknown          57  2020-04-01
47   Female       22485  2020-04-02
48     Male       27155  2020-04-02
49  Unknown          67  2020-04-02
50   Female       25536  2020-04-03
51     Male       30672  2020-04-03
52  Unknown          81  2020-04-03
53   Female       27660  2020-04-04
54     Male       33100  2020-04-04
55  Unknown          90  2020-04-04
56   Female       29607  2020-04-05
57     Male       35256  2020-04-05
58  Unknown          92  2020-04-05
59   Female       31319  2020-04-06
60     Male       37361  2020-04-06
61  Unknown          96  2020-04-06

In [113]:
sex_cases.loc[:,'Sex'] = sex_cases.loc[:,'Sex'].map(fixvalues)

In [290]:
sex_deaths

Sex  Underlying Conditions  No Underlying Conditions  \
0    Female                      0                         0   
1      Male                      0                         0   
2    Female                      0                         0   
3      Male                      0                         0   
4    Female                      0                         0   
5      Male                      0                         0   
6    Female                      0                         0   
7      Male                      0                         0   
8    Female                      0                         0   
9      Male                      0                         0   
10   Female                    155                         5   
11     Male                    218                         9   
12   Female                    213                         6   
13     Male                    314                        11   
14   Female                    246                         5   
15     Male                    378                         8   
16   Female                    270                         4   
17     Male                    433                        10   
18  Unknown                      1                         0   
19   Female                    289                         4   
20     Male                    481                        10   
21  Unknown                      1                         0   
22   Female                    373                         4   
23     Male                    620                        14   
24  Unknown                      1                         0   
25   Female                    432                         6   
26     Male                    711                        18   
27  Unknown                      2                         0   
28   Female                    490                         5   
29     Male                    793                        23   
30  Unknown                      2                         0   
31   Female                    559                         7   
32     Male                    937                        28   
33  Unknown                      2                         0   
34   Female                    573                         7   
35     Male                    960                        31   
36  Unknown                      2                         0   
37   Female                    632                        11   
38     Male                   1059                        37   
39  Unknown                      2                         0   

    Underlying Conditions Pending  Total Deaths        Date  
0                               0            39  2020-03-22  
1                               0            60  2020-03-22  
2                               0            47  2020-03-23  
3                               0            78  2020-03-23  
4                               0            74  2020-03-24  
5                               0           118  2020-03-24  
6                               0           111  2020-03-25  
7                               0           169  2020-03-25  
8                               0           149  2020-03-26  
9                               0           216  2020-03-26  
10                             21           181  2020-03-27  
11                             42           269  2020-03-27  
12                             42           261  2020-03-28  
13                             86           411  2020-03-28  
14                             51           302  2020-03-29  
15                             88           474  2020-03-29  
16                             70           344  2020-03-30  
17                            126           569  2020-03-30  
18                              0             1  2020-03-30  
19                            108           401  2020-03-31  
20                            203           694  2020-03-31  
21                 

In [288]:
problem = sex_deaths['Sex']
uniquevalues = list(problem.unique())
for idx, elmt in enumerate(uniquevalues):
    print(idx, elmt)
    
correction = {'good':uniquevalues[0:2],'bad':uniquevalues[2:4]}
correction

0 Female
1 Male
2 emale
3 ale
4 Unknown


{'good': ['Female', 'Male'], 'bad': ['emale', 'ale']}

In [289]:
sex_deaths.loc[:,'Sex'] = sex_deaths.loc[:,'Sex'].map(fixvalues)

In [117]:
#sex_death['Sex'].apply(lambda x: temp_dict['emale'] if x == 'emale' else temp_dict['ale'] if x == 'ale' else x)

In [118]:
borough_hosp

Borough  Ever Hospitalized Cases  Total Cases        Date
0           Bronx                      600         2505  2020-03-24
1        Brooklyn                      637         4407  2020-03-24
2       Manhattan                      417         3013  2020-03-24
3          Queens                     1037         4667  2020-03-24
4   Staten Island                      189          999  2020-03-24
5         Unknown                        3            6  2020-03-24
6           Bronx                      802         3542  2020-03-25
7        Brooklyn                      919         5232  2020-03-25
8       Manhattan                      584         3616  2020-03-25
9          Queens                     1363         6420  2020-03-25
10  Staten Island                      251         1166  2020-03-25
11        Unknown                        3           35  2020-03-25
12          Bronx                     1071         4243  2020-03-26
13       Brooklyn                      968         6095  2020-03-26
14      Manhattan                      729         4046  2020-03-26
15         Queens                     1659         7362  2020-03-26
16  Staten Island                      283         1330  2020-03-26
17        Unknown                       10           36  2020-03-26
18          Bronx                     1200         4880  2020-03-27
19       Brooklyn                      989         7091  2020-03-27
20      Manhattan                      709         4627  2020-03-27
21         Queens                     1810         8529  2020-03-27
22  Staten Island                      321         1534  2020-03-27
23        Unknown                       10           36  2020-03-27
24          Bronx                     1433         5752  2020-03-28
25       Brooklyn                     1372         8129  2020-03-28
26      Manhattan                      916         5237  2020-03-28
27         Queens                     2181         9831  2020-03-28
28  Staten Island                      374         1781  2020-03-28
29        Unknown                       11           35  2020-03-28
30          Bronx                     1729         6250  2020-03-29
31       Brooklyn                     1617         8887  2020-03-29
32      Manhattan                     1074         5582  2020-03-29
33         Queens                     2536        10737  2020-03-29
34  Staten Island                      444         1984  2020-03-29
35        Unknown                       10           34  2020-03-29
36          Bronx                     1880         6925  2020-03-30
37       Brooklyn                     1661        10171  2020-03-30
38      Manhattan                     1075         6060  2020-03-30
39         Queens                     2650        12756  2020-03-30
40  Staten Island                      465         2140  2020-03-30
41        Unknown                       10           35  2020-03-30
42          Bronx                     2056         7815  2020-03-31
43       Brooklyn                     1901        11160  2020-03-31
44      Manhattan                     1130         6538  2020-03-31
45         Queens                     2954        13869  2020-03-31
46  Staten Island                      499         2354  2020-03-31
47        Unknown                        9           35  2020-03-31
48          Bronx                     2414         8608  2020-04-01
49       Brooklyn                     2229        12274  2020-04-01
50      Manhattan                     1287         7021  2020-04-01
51         Queens                     3287        15217  2020-04-01
52  Staten Island                      549         2552  2020-04-01
53        Unknown                        9           35  2020-04-01
54          Bronx                     2657         9344  2020-04-02
55       Brooklyn                     2444        13290  2020-04-02
56      Manhattan                     1337         7397  2020-04-02
57         Queens                     3531        16819  2020-04-0

In [124]:
borough_deaths

Borough  Underlying Conditions  No Underlying Conditions  \
0           Bronx                      0                         0   
1        Brooklyn                      0                         0   
2       Manhattan                      0                         0   
3          Queens                      0                         0   
4   Staten Island                      0                         0   
5           Bronx                      0                         0   
6        Brooklyn                      0                         0   
7       Manhattan                      0                         0   
8          Queens                      0                         0   
9   Staten Island                      0                         0   
10          Bronx                      0                         0   
11       Brooklyn                      0                         0   
12      Manhattan                      0                         0   
13         Queens                      0                         0   
14  Staten Island                      0                         0   
15          Bronx                      0                         0   
16       Brooklyn                      0                         0   
17      Manhattan                      0                         0   
18         Queens                      0                         0   
19  Staten Island                      0                         0   
20          Bronx                      0                         0   
21       Brooklyn                      0                         0   
22      Manhattan                      0                         0   
23         Queens                      0                         0   
24  Staten Island                      0                         0   
25          Bronx                     92                         0   
26       Brooklyn                     77                         5   
27      Manhattan                     53                         2   
28         Queens                    125                         6   
29  Staten Island                     26                         1   
30          Bronx                    142                         1   
31       Brooklyn                    112                         5   
32      Manhattan                     71                         2   
33         Queens                    168                         9   
34  Staten Island                     33                         0   
35        Unknown                      1                         0   
36          Bronx                    176                         0   
37       Brooklyn                    136                         5   
38      Manhattan                     82                         1   
39         Queens                    194                         7   
40  Staten Island                     35                         0   
41        Unknown                      1                         0   
42          Bronx                    198                         1   
43       Brooklyn                    153                         4   
44      Manhattan                     90                         1   
45         Queens                    222                         8   
46  Staten Island                     40                         0   
47        Unknown                      1                         0   
48          Bronx                    230                         1   
49       Brooklyn                    163                         4   
50      Manhattan                     93                         1   
51         Queens                    240                         8   
52  Staten Island                     44                         0   
53        Unknown                      1                         0   
54          Bronx                    305                         4   
55       Brooklyn                    207                         4   
56      Manhattan   

In [123]:
borough_deaths.loc[:,'Borough']=borough_deaths.loc[:,'Borough'].map(fixvalues)

In [130]:
borough_cases

Borough Total Cases        Date
0            Bronx         106  2020-03-17
1         Brooklyn         236  2020-03-17
2        Manhattan         290  2020-03-17
3           Queens         248  2020-03-17
4    Staten Island          42  2020-03-17
5          Unknown           1  2020-03-17
6            Bronx         303  2020-03-18
7         Brooklyn         505  2020-03-18
8        Manhattan         590  2020-03-18
9           Queens         533  2020-03-18
10   Staten Island          77  2020-03-18
11         Unknown           1  2020-03-18
12           Bronx         496  2020-03-19
13        Brooklyn        1195  2020-03-19
14       Manhattan        1038  2020-03-19
15          Queens        1042  2020-03-19
16   Staten Island         179  2020-03-19
17         Unknown           4  2020-03-19
18           Bronx         736  2020-03-20
19        Brooklyn        1740  2020-03-20
20       Manhattan        1402  2020-03-20
21          Queens        1514  2020-03-20
22   Staten Island         285  2020-03-20
23         Unknown           6  2020-03-20
24           Bronx        1071  2020-03-21
25        Brooklyn        2484  2020-03-21
26       Manhattan        1863  2020-03-21
27          Queens        2254  2020-03-21
28   Staten Island         437  2020-03-21
29         Unknown           6  2020-03-21
30           Bronx        1564  2020-03-22
31        Brooklyn        3154  2020-03-22
32       Manhattan        2324  2020-03-22
33          Queens        3050  2020-03-22
34   Staten Island         666  2020-03-22
35         Unknown           6  2020-03-22
36           Bronx        1999  2020-03-23
37        Brooklyn        3742  2020-03-23
38       Manhattan        2646  2020-03-23
39          Queens        3848  2020-03-23
40   Staten Island         877  2020-03-23
41         Unknown           7  2020-03-23
42           Bronx        2505  2020-03-24
43        Brooklyn        4407  2020-03-24
44       Manhattan        3013  2020-03-24
45          Queens        4667  2020-03-24
46   Staten Island         999  2020-03-24
47         Unknown           6  2020-03-24
48           Bronx        3542  2020-03-25
49        Brooklyn        5232  2020-03-25
50       Manhattan        3616  2020-03-25
51          Queens        6420  2020-03-25
52   Staten Island        1166  2020-03-25
53         Unknown          35  2020-03-25
54           Bronx        4243  2020-03-26
55        Brooklyn        6095  2020-03-26
56       Manhattan        4046  2020-03-26
57          Queens        7362  2020-03-26
58   Staten Island        1330  2020-03-26
59         Unknown          36  2020-03-26
60           Bronx        4880  2020-03-27
61        Brooklyn        7091  2020-03-27
62       Manhattan        4627  2020-03-27
63          Queens        8529  2020-03-27
64   Staten Island        1534  2020-03-27
65         Unknown          36  2020-03-27
66           Bronx        5752  2020-03-28
67        Brooklyn        8129  2020-03-28
68       Manhattan        5237  2020-03-28
69          Queens        9831  2020-03-28
70   Staten Island        1781  2020-03-28
71         Unknown          35  2020-03-28
72           Bronx        6250  2020-03-29
73        Brooklyn        8887  2020-03-29
74       Manhattan        5582  2020-03-29
75          Queens       10737  2020-03-29
76   Staten Island        1984  2020-03-29
77         Unknown          34  2020-03-29
78           Bronx        6925  2020-03-30
79        Brooklyn       10171  2020-03-30
80       Manhattan        6060  2020-03-30
81          Queens       12756  2020-03-30
82   Staten Island        2140  2020-03-30
83         Unknown          35  2020-03-30
84           Bronx        7814  2020-03-31
85        Brooklyn       11160  2020-03-31
86       Manhattan        6539  2020-03-31
87          Queens       13869  2020-03-31
88   Staten Island        2354  2020-03-31
89         Unknown          35  2020-03-31
90           Bronx        8607  2020-04-01
91        Brooklyn       12274  2020-04-01
92       Man

In [128]:
borough_cases.loc[:,'Borough']=borough_cases.loc[:,'Borough'].map(fixvalues)

In [135]:
cases50

Age 50 and over Total Cases        Date
0              Yes           0  2020-03-17
1               No           0  2020-03-17
2              Yes         927  2020-03-18
3               No        1081  2020-03-18
4              Yes        1726  2020-03-19
5               No        2225  2020-03-19
6              Yes        2442  2020-03-20
7               No        3237  2020-03-20
8              Yes        3431  2020-03-21
9               No        4679  2020-03-21
10             Yes        4585  2020-03-22
11              No        6174  2020-03-22
12             Yes        5596  2020-03-23
13              No        7513  2020-03-23
14             Yes        6787  2020-03-24
15              No        8801  2020-03-24
16             Yes        8920  2020-03-25
17              No       11051  2020-03-25
18             Yes       10452  2020-03-26
19              No       12619  2020-03-26
20             Yes       12209  2020-03-27
21              No       14429  2020-03-27
22             Yes       14279  2020-03-28
23              No       16416  2020-03-28
24             Yes       15706  2020-03-29
25              No       17697  2020-03-29
26             Yes       17958  2020-03-30
27              No       20034  2020-03-30
28             Yes       19973  2020-03-31
29              No       21701  2020-03-31
30             Yes       22061  2020-04-01
31              No       23538  2020-04-01
32             Yes       24129  2020-04-02
33              No       25468  2020-04-02
34             Yes       27939  2020-04-03
35              No       28227  2020-04-03
36             Yes       30528  2020-04-04
37              No       30198  2020-04-04
38             Yes       32802  2020-04-05
39              No       32016  2020-04-05
40             Yes       34868  2020-04-06
41              No       33773  2020-04-06

In [133]:
problem = cases50['Age 50 and over']
uniquevalues = list(problem.unique())
for idx, elmt in enumerate(uniquevalues):
    print(idx, elmt)
    
correction = {'good':uniquevalues[0:2],'bad':uniquevalues[2:4]}
correction

0 Yes
1 No
2 es
3 o


{'good': ['Yes', 'No'], 'bad': ['es', 'o']}

In [134]:
cases50.loc[:,'Age 50 and over']=cases50.loc[:,'Age 50 and over'].map(fixvalues)

In [260]:
#output to csvs for backup.

catdfs = [age_hosp,sex_hosp,borough_hosp,age_deaths,sex_deaths,borough_deaths,age_cases,cases50,sex_cases,borough_cases]
catdfsnames = ['age_hosp.csv','sex_hosp.csv','borough_hosp.csv','age_deaths.csv','sex_deaths.csv','borough_deaths.csv','age_cases.csv','cases50.csv','sex_cases.csv','borough_cases.csv']

for i in range(len(catdfs)):
    df.to_csv(folderpath /  catdfsnames[i])


# agenew.to_csv(folderpath / 'age.csv', index = False)
# sexnew.to_csv(folderpath / 'sex.csv', index = False)
# boroughnew.to_csv(folderpath / 'borough.csv', index = False)
# cases50current.to_csv(folderpath / 'cases50.csv', index = False)

In [291]:
#sex_deaths.to_csv(folderpath /  'sex_deaths.csv')

### Once confirmed, combine various metrics dataframes into the same categories

In [137]:
#combines  several of the new dataframes into one
def mergedfs(cases,leftdf,rightdf,on):
    leftdf = leftdf.drop(['Total Cases','Date'], axis = 1)
    leftdf = pd.merge(leftdf,cases,on = on)
    temp = pd.merge(leftdf,rightdf, how='left', on = on)
    temp.rename({'Date_x':'Date'},axis = 1, inplace = True)
    temp.drop(['Date_y'], axis=1, inplace = True)
    temp = temp.fillna(0)
    return temp
    

In [144]:
cases = age_cases    
leftdf = age_hosp
rightdf = age_deaths
on = 'Age Group'

leftdf = leftdf.drop(['Total Cases','Date'], axis = 1)
leftdf = pd.merge(cases,leftdf,on = on)
leftdf
#temp = pd.merge(leftdf,rightdf, how='left', on = on)
#temp.rename({'Date_x':'Date'},axis = 1, inplace = True)
#temp.drop(['Date_y'], axis=1, inplace = True)
#temp = temp.fillna(0)

Age Group Total Cases        Date  Ever Hospitalized Cases
0         Unknown           1  2020-03-18                        0
1         Unknown           1  2020-03-18                        0
2         Unknown           1  2020-03-18                        0
3         Unknown           1  2020-03-18                        0
4         Unknown           1  2020-03-18                        0
...           ...         ...         ...                      ...
1395  75 and over        7345  2020-04-06                     2423
1396  75 and over        7345  2020-04-06                     2745
1397  75 and over        7345  2020-04-06                     3018
1398  75 and over        7345  2020-04-06                     3423
1399  75 and over        7345  2020-04-06                     3751

[1400 rows x 4 columns]

In [140]:
age_hosp

Age Group  Ever Hospitalized Cases  Total Cases        Date
0       0 to 17                       28          384  2020-03-24
1      18 to 44                      629         7094  2020-03-24
2      45 to 64                     1061         5194  2020-03-24
3      65 to 74                      550         1689  2020-03-24
4   75 and over                      615         1227  2020-03-24
5       Unknown                        0            9  2020-03-24
6       0 to 17                       35          446  2020-03-25
7      18 to 44                      837         8880  2020-03-25
8      45 to 64                     1470         6786  2020-03-25
9      65 to 74                      736         2226  2020-03-25
10  75 and over                      844         1633  2020-03-25
11      Unknown                        0           40  2020-03-25
12      0 to 17                       46          495  2020-03-26
13     18 to 44                      950        10145  2020-03-26
14     45 to 64                     1749         7869  2020-03-26
15     65 to 74                      946         2627  2020-03-26
16  75 and over                     1029         1935  2020-03-26
17      Unknown                        0           41  2020-03-26
18      0 to 17                       47          543  2020-03-27
19     18 to 44                      971        11617  2020-03-27
20     45 to 64                     1886         9158  2020-03-27
21     65 to 74                     1032         3034  2020-03-27
22  75 and over                     1103         2286  2020-03-27
23      Unknown                        0           59  2020-03-27
24      0 to 17                       58          591  2020-03-28
25     18 to 44                     1224        13213  2020-03-28
26     45 to 64                     2350        10596  2020-03-28
27     65 to 74                     1267         3571  2020-03-28
28  75 and over                     1388         2724  2020-03-28
29      Unknown                        0           70  2020-03-28
30      0 to 17                       67          619  2020-03-29
31     18 to 44                     1459        14233  2020-03-29
32     45 to 64                     2765        11577  2020-03-29
33     65 to 74                     1499         3954  2020-03-29
34  75 and over                     1620         3020  2020-03-29
35      Unknown                        0           71  2020-03-29
36      0 to 17                       72          714  2020-03-30
37     18 to 44                     1448        16028  2020-03-30
38     45 to 64                     2887        13344  2020-03-30
39     65 to 74                     1612         4496  2020-03-30
40  75 and over                     1722         3410  2020-03-30
41      Unknown                        0           95  2020-03-30
42      0 to 17                       74          757  2020-03-31
43     18 to 44                     1532        17347  2020-03-31
44     45 to 64                     3195        14689  2020-03-31
45     65 to 74                     1826         5015  2020-03-31
46  75 and over                     1922         3866  2020-03-31
47      Unknown                        0           97  2020-03-31
48      0 to 17                       83          816  2020-04-01
49     18 to 44                     1711        18767  2020-04-01
50     45 to 64                     3658        16104  2020-04-01
51     65 to 74                     2114         5584  2020-04-01
52  75 and over                     2209         4328  2020-04-01
53      Unknown                        0          108  2020-04-01
54      0 to 17                       88          881  2020-04-02
55     18 to 44                     1817        20313  2020-04-02
56     45 to 64                     3962        17497  2020-04-02
57     65 to 74                     2300         6095  2020-04-02
58  75 and over                     2423         4811  2020-04-02
59      Unknown                        0      

In [151]:
dates_dict['datescases'][7][:-2]

'03242020'

In [211]:
del age_stack

NameError: name 'age_stack' is not defined

In [212]:
age_stack = age_cases[age_cases['Date'] < datetime.datetime.strptime('03242020', '%m%d%Y').date()]
age_stack.loc[:,'Ever Hospitalized Cases'] = 0
#age_stack = age_stack[['Age Group','Ever Hospitalized Cases','Total Cases','Date']]

In [213]:
age_stack = age_stack[['Age Group','Ever Hospitalized Cases','Total Cases','Date']]

In [216]:
age_hosp = pd.concat([age_stack,age_hosp], axis = 0, sort = False).reset_index(drop = True)

In [218]:
age_hosp

Age Group  Ever Hospitalized Cases Total Cases        Date
0         0 to 4                        0           0  2020-03-17
1        5 to 17                        0          25  2020-03-17
2       18 to 49                        0         457  2020-03-17
3       50 to 64                        0         211  2020-03-17
4    65 and over                        0         230  2020-03-17
5         0 to 4                        0           5  2020-03-18
6        5 to 17                        0          42  2020-03-18
7       18 to 49                        0        1034  2020-03-18
8       50 to 64                        0         463  2020-03-18
9    65 and over                        0         464  2020-03-18
10       Unknown                        0           1  2020-03-18
11        0 to 4                        0          14  2020-03-19
12       5 to 17                        0          94  2020-03-19
13      18 to 49                        0        2117  2020-03-19
14      50 to 64                        0         926  2020-03-19
15   65 and over                        0         800  2020-03-19
16       Unknown                        0           3  2020-03-19
17        0 to 4                        0          26  2020-03-20
18       5 to 17                        0         131  2020-03-20
19      18 to 49                        0        3080  2020-03-20
20      50 to 64                        0        1330  2020-03-20
21   65 and over                        0        1112  2020-03-20
22       Unknown                        0           4  2020-03-20
23        0 to 4                        0          34  2020-03-21
24       5 to 17                        0         189  2020-03-21
25      18 to 49                        0        4456  2020-03-21
26      50 to 64                        0        1929  2020-03-21
27   65 and over                        0        1502  2020-03-21
28       Unknown                        0           5  2020-03-21
29       0 to 17                        0         300  2020-03-22
30      18 to 44                        0        4965  2020-03-22
31      45 to 64                        0        3519  2020-03-22
32      65 to 74                        0        1150  2020-03-22
33   75 and over                        0         825  2020-03-22
34       Unknown                        0           5  2020-03-22
35       0 to 17                        0         341  2020-03-23
36      18 to 44                        0        6035  2020-03-23
37      45 to 64                        0        4343  2020-03-23
38      65 to 74                        0        1402  2020-03-23
39   75 and over                        0         988  2020-03-23
40       Unknown                        0          10  2020-03-23
41       0 to 17                       28         384  2020-03-24
42      18 to 44                      629        7094  2020-03-24
43      45 to 64                     1061        5194  2020-03-24
44      65 to 74                      550        1689  2020-03-24
45   75 and over                      615        1227  2020-03-24
46       Unknown                        0           9  2020-03-24
47       0 to 17                       35         446  2020-03-25
48      18 to 44                      837        8880  2020-03-25
49      45 to 64                     1470        6786  2020-03-25
50      65 to 74                      736        2226  2020-03-25
51   75 and over                      844        1633  2020-03-25
52       Unknown                        0          40  2020-03-25
53       0 to 17                       46         495  2020-03-26
54      18 to 44                      950       10145  2020-03-26
55      45 to 64                     1749        7869  2020-03-26
56      65 to 74                      946        2627  2020-03-26
57   75 and over                     1029        1935  2020-03-26
58       Unknown                        0          41  2020-03-26
59       0 to 17                       47      

In [253]:
age_hosp['Total Cases'].sum()

620365

In [219]:
age_deaths

Age Group  Underlying Conditions  No Underlying Conditions  \
0       0 to 17                      0                         0   
1      18 to 44                      0                         0   
2      45 to 64                      0                         0   
3      65 to 74                      0                         0   
4   75 and over                      0                         0   
5       0 to 17                      0                         0   
6      18 to 44                      0                         0   
7      45 to 64                      0                         0   
8      65 to 74                      0                         0   
9   75 and over                      0                         0   
10      0 to 17                      0                         0   
11     18 to 44                      0                         0   
12     45 to 64                      0                         0   
13     65 to 74                      0                         0   
14  75 and over                      0                         0   
15      0 to 17                      0                         0   
16     18 to 44                      0                         0   
17     45 to 64                      0                         0   
18     65 to 74                      0                         0   
19  75 and over                      0                         0   
20      0 to 17                      0                         0   
21     18 to 44                      0                         0   
22     45 to 64                      0                         0   
23     65 to 74                      0                         0   
24  75 and over                      0                         0   
25      0 to 17                      0                         0   
26     18 to 44                     15                         1   
27     45 to 64                     79                         7   
28     65 to 74                     90                         2   
29  75 and over                    189                         4   
30      0 to 17                      0                         0   
31     18 to 44                     23                         1   
32     45 to 64                    125                         8   
33     65 to 74                    125                         2   
34  75 and over                    254                         6   
35      0 to 17                      1                         0   
36     18 to 44                     28                         1   
37     45 to 64                    140                         8   
38     65 to 74                    152                         1   
39  75 and over                    303                         3   
40      0 to 17                      1                         0   
41     18 to 44                     39                         2   
42     45 to 64                    170                         9   
43     65 to 74                    167                         0   
44  75 and over                    327                         3   
45      0 to 17                      1                         0   
46     18 to 44                     45                         2   
47     45 to 64                    187                         9   
48     65 to 74                    187                         0   
49  75 and over                    351                         3   
50      0 to 17                      1                         0   
51     18 to 44                     51                         4   
52     45 to 64                    254                         9   
53     65 to 74                    257                         1   
54  75 and over                    431                         4   
55      0 to 17                      1                         0   
56     18 to 44                     64                         7   
57     45 to 64                    304                        

In [257]:
age_deaths['Total Deaths'].sum()

17236

In [227]:
print(age_deaths.shape)
print(age_hosp.shape)

(80, 6)
(125, 4)


In [270]:
age_combined = pd.merge(age_hosp,age_deaths, how='left', on = ['Age Group','Date']).fillna(0)
agecols = list(age_combined.columns)
agecols.remove('Date')
agecols.append('Date')
age_combined = age_combined[agecols]
age_combined

Age Group  Ever Hospitalized Cases  Total Cases  Underlying Conditions  \
0         0 to 4                        0            0                    0.0   
1        5 to 17                        0           25                    0.0   
2       18 to 49                        0          457                    0.0   
3       50 to 64                        0          211                    0.0   
4    65 and over                        0          230                    0.0   
5         0 to 4                        0            5                    0.0   
6        5 to 17                        0           42                    0.0   
7       18 to 49                        0         1034                    0.0   
8       50 to 64                        0          463                    0.0   
9    65 and over                        0          464                    0.0   
10       Unknown                        0            1                    0.0   
11        0 to 4                        0           14                    0.0   
12       5 to 17                        0           94                    0.0   
13      18 to 49                        0         2117                    0.0   
14      50 to 64                        0          926                    0.0   
15   65 and over                        0          800                    0.0   
16       Unknown                        0            3                    0.0   
17        0 to 4                        0           26                    0.0   
18       5 to 17                        0          131                    0.0   
19      18 to 49                        0         3080                    0.0   
20      50 to 64                        0         1330                    0.0   
21   65 and over                        0         1112                    0.0   
22       Unknown                        0            4                    0.0   
23        0 to 4                        0           34                    0.0   
24       5 to 17                        0          189                    0.0   
25      18 to 49                        0         4456                    0.0   
26      50 to 64                        0         1929                    0.0   
27   65 and over                        0         1502                    0.0   
28       Unknown                        0            5                    0.0   
29       0 to 17                        0          300                    0.0   
30      18 to 44                        0         4965                    0.0   
31      45 to 64                        0         3519                    0.0   
32      65 to 74                        0         1150                    0.0   
33   75 and over                        0          825                    0.0   
34       Unknown                        0            5                    0.0   
35       0 to 17                        0          341                    0.0   
36      18 to 44                        0         6035                    0.0   
37      45 to 64                        0         4343                    0.0   
38      65 to 74                        0         1402                    0.0   
39   75 and over                        0          988                    0.0   
40       Unknown                        0           10                    0.0   
41       0 to 17                       28          384                    0.0   
42      18 to 44                      629         7094                    0.0   
43      45 to 64                     1061         5194                    0.0   
44      65 to 74                      550         1689                    0.0   
45   75 and over                      615         1227                    0.0   
46       Unknown                        0            9                    0.0   
47       0 to 17                       35          446                    0.0   
48      18 to 44                      

In [272]:
age_combined['Total Cases'].sum()

620365

In [273]:
age_combined['Total Deaths'].sum()

17236.0

In [285]:
sex_hosp = pd.merge(sex_cases,sex_hosp, how = 'left', on = ['Sex','Date']).fillna(0).drop(['Total Cases_y'],axis = 1).rename({'Total Cases_x':'Total Cases'}, axis = 1)
sex_hosp

Sex  Total Cases        Date  Ever Hospitalized Cases
0    Female          378  2020-03-17                      0.0
1      Male          536  2020-03-17                      0.0
2    Female          850  2020-03-18                      0.0
3      Male         1155  2020-03-18                      0.0
4   Unknown            4  2020-03-18                      0.0
5    Female         1630  2020-03-19                      0.0
6      Male         2315  2020-03-19                      0.0
7   Unknown            9  2020-03-19                      0.0
8    Female         2317  2020-03-20                      0.0
9      Male         3355  2020-03-20                      0.0
10  Unknown           11  2020-03-20                      0.0
11   Female         3318  2020-03-21                      0.0
12     Male         4787  2020-03-21                      0.0
13  Unknown           10  2020-03-21                      0.0
14   Female         4522  2020-03-22                      0.0
15     Male         6227  2020-03-22                      0.0
16  Unknown           15  2020-03-22                      0.0
17   Female         5612  2020-03-23                      0.0
18     Male         7490  2020-03-23                      0.0
19  Unknown           17  2020-03-23                      0.0
20   Female         6736  2020-03-24                   1191.0
21     Male         8838  2020-03-24                   1691.0
22  Unknown           23  2020-03-24                      1.0
23   Female         8655  2020-03-25                   1628.0
24     Male        11325  2020-03-25                   2293.0
25  Unknown           31  2020-03-25                      1.0
26   Female        10124  2020-03-26                   1918.0
27     Male        12948  2020-03-26                   2801.0
28  Unknown           40  2020-03-26                      1.0
29   Female        11792  2020-03-27                   2035.0
30     Male        14863  2020-03-27                   3003.0
31  Unknown           42  2020-03-27                      1.0
32   Female        13592  2020-03-28                   2545.0
33     Male        17133  2020-03-28                   3741.0
34  Unknown           40  2020-03-28                      1.0
35   Female        14837  2020-03-29                   3001.0
36     Male        18593  2020-03-29                   4408.0
37  Unknown           44  2020-03-29                      1.0
38   Female        16920  2020-03-30                   3130.0
39     Male        21120  2020-03-30                   4610.0
40  Unknown           47  2020-03-30                      1.0
41   Female        18677  2020-03-31                   3455.0
42     Male        23043  2020-03-31                   5093.0
43  Unknown           51  2020-03-31                      1.0
44   Female        20522  2020-04-01                   3980.0
45     Male        25128  2020-04-01                   5792.0
46  Unknown           57  2020-04-01                      3.0
47   Female        22485  2020-04-02                   4292.0
48     Male        27155  2020-04-02                   6295.0
49  Unknown           67  2020-04-02                      3.0
50   Female        25536  2020-04-03                   4765.0
51     Male        30672  2020-04-03                   6970.0
52  Unknown           81  2020-04-03                      4.0
53   Female        27660  2020-04-04                   5116.0
54     Male        33100  2020-04-04                   7596.0
55  Unknown           90  2020-04-04                      4.0
56   Female        29607  2020-04-05                   5714.0
57     Male        35256  2020-04-05                   8487.0
58  Unknown           92  2020-04-05                      4.0
59   Female        31319  2020-04-06                   6166.0
60     Male        37361  2020-04-06                   9163.0
61  Unknown           96  2020-04-06                      4.0

In [283]:
def checktotals(series,combined,metric):
    print('Sum of first is ',series[metric].sum())
    print('Sum of second is ',combined[metric].sum())
    print('Are they equal?')
    return series[metric].sum() == combined[metric].sum()
    

In [286]:
#ascertain that sums are intact
checktotals(sex_cases,sex_hosp,'Total Cases')

Sum of first is  620356
Sum of second is  620356
Are they equal?


True

In [295]:
sex_combined = pd.merge(sex_hosp,sex_deaths,how='left',on=['Sex','Date']).fillna(0)
sexcols = list(sex_combined.columns)
sexcols.remove('Date')
sexcols.append('Date')
sex_combined = sex_combined[sexcols]
sex_combined

Sex  Total Cases  Ever Hospitalized Cases  Underlying Conditions  \
0    Female          378                      0.0                    0.0   
1      Male          536                      0.0                    0.0   
2    Female          850                      0.0                    0.0   
3      Male         1155                      0.0                    0.0   
4   Unknown            4                      0.0                    0.0   
5    Female         1630                      0.0                    0.0   
6      Male         2315                      0.0                    0.0   
7   Unknown            9                      0.0                    0.0   
8    Female         2317                      0.0                    0.0   
9      Male         3355                      0.0                    0.0   
10  Unknown           11                      0.0                    0.0   
11   Female         3318                      0.0                    0.0   
12     Male         4787                      0.0                    0.0   
13  Unknown           10                      0.0                    0.0   
14   Female         4522                      0.0                    0.0   
15     Male         6227                      0.0                    0.0   
16  Unknown           15                      0.0                    0.0   
17   Female         5612                      0.0                    0.0   
18     Male         7490                      0.0                    0.0   
19  Unknown           17                      0.0                    0.0   
20   Female         6736                   1191.0                    0.0   
21     Male         8838                   1691.0                    0.0   
22  Unknown           23                      1.0                    0.0   
23   Female         8655                   1628.0                    0.0   
24     Male        11325                   2293.0                    0.0   
25  Unknown           31                      1.0                    0.0   
26   Female        10124                   1918.0                    0.0   
27     Male        12948                   2801.0                    0.0   
28  Unknown           40                      1.0                    0.0   
29   Female        11792                   2035.0                  155.0   
30     Male        14863                   3003.0                  218.0   
31  Unknown           42                      1.0                    0.0   
32   Female        13592                   2545.0                  213.0   
33     Male        17133                   3741.0                  314.0   
34  Unknown           40                      1.0                    0.0   
35   Female        14837                   3001.0                  246.0   
36     Male        18593                   4408.0                  378.0   
37  Unknown           44                      1.0                    0.0   
38   Female        16920                   3130.0                  270.0   
39     Male        21120                   4610.0                  433.0   
40  Unknown           47                      1.0                    1.0   
41   Female        18677                   3455.0                  289.0   
42     Male        23043                   5093.0                  481.0   
43  Unknown           51                      1.0                    1.0   
44   Female        20522                   3980.0                  373.0   
45     Male        25128                   5792.0                  620.0   
46  Unknown           57                      3.0                    1.0   
47   Female        22485                   4292.0                  432.0   
48     Male        27155                   6295.0                  711.0   
49  Unknown           67                      3.0                    2.0   
50   Female        25536                   4765.0                  490.0   
51     Male        30672                   6970.0       

In [297]:
checktotals(sex_deaths,sex_combined,'Total Deaths')
checktotals(sex_hosp,sex_combined,'Ever Hospitalized Cases')


Sum of first is  17236
Sum of second is  17236.0
Are they equal?
Sum of first is  120909.0
Sum of second is  120909.0
Are they equal?


True

In [292]:
sex_deaths

Sex  Underlying Conditions  No Underlying Conditions  \
0    Female                      0                         0   
1      Male                      0                         0   
2    Female                      0                         0   
3      Male                      0                         0   
4    Female                      0                         0   
5      Male                      0                         0   
6    Female                      0                         0   
7      Male                      0                         0   
8    Female                      0                         0   
9      Male                      0                         0   
10   Female                    155                         5   
11     Male                    218                         9   
12   Female                    213                         6   
13     Male                    314                        11   
14   Female                    246                         5   
15     Male                    378                         8   
16   Female                    270                         4   
17     Male                    433                        10   
18  Unknown                      1                         0   
19   Female                    289                         4   
20     Male                    481                        10   
21  Unknown                      1                         0   
22   Female                    373                         4   
23     Male                    620                        14   
24  Unknown                      1                         0   
25   Female                    432                         6   
26     Male                    711                        18   
27  Unknown                      2                         0   
28   Female                    490                         5   
29     Male                    793                        23   
30  Unknown                      2                         0   
31   Female                    559                         7   
32     Male                    937                        28   
33  Unknown                      2                         0   
34   Female                    573                         7   
35     Male                    960                        31   
36  Unknown                      2                         0   
37   Female                    632                        11   
38     Male                   1059                        37   
39  Unknown                      2                         0   

    Underlying Conditions Pending  Total Deaths        Date  
0                               0            39  2020-03-22  
1                               0            60  2020-03-22  
2                               0            47  2020-03-23  
3                               0            78  2020-03-23  
4                               0            74  2020-03-24  
5                               0           118  2020-03-24  
6                               0           111  2020-03-25  
7                               0           169  2020-03-25  
8                               0           149  2020-03-26  
9                               0           216  2020-03-26  
10                             21           181  2020-03-27  
11                             42           269  2020-03-27  
12                             42           261  2020-03-28  
13                             86           411  2020-03-28  
14                             51           302  2020-03-29  
15                             88           474  2020-03-29  
16                             70           344  2020-03-30  
17                            126           569  2020-03-30  
18                              0             1  2020-03-30  
19                            108           401  2020-03-31  
20                            203           694  2020-03-31  
21                 

In [ ]:
# borough_combined = mergedfs(borough_cases,borough_hosp,borough_death,'Borough')
# borough_combined

In [298]:
borough_hosp

Borough  Ever Hospitalized Cases  Total Cases        Date
0           Bronx                      600         2505  2020-03-24
1        Brooklyn                      637         4407  2020-03-24
2       Manhattan                      417         3013  2020-03-24
3          Queens                     1037         4667  2020-03-24
4   Staten Island                      189          999  2020-03-24
5         Unknown                        3            6  2020-03-24
6           Bronx                      802         3542  2020-03-25
7        Brooklyn                      919         5232  2020-03-25
8       Manhattan                      584         3616  2020-03-25
9          Queens                     1363         6420  2020-03-25
10  Staten Island                      251         1166  2020-03-25
11        Unknown                        3           35  2020-03-25
12          Bronx                     1071         4243  2020-03-26
13       Brooklyn                      968         6095  2020-03-26
14      Manhattan                      729         4046  2020-03-26
15         Queens                     1659         7362  2020-03-26
16  Staten Island                      283         1330  2020-03-26
17        Unknown                       10           36  2020-03-26
18          Bronx                     1200         4880  2020-03-27
19       Brooklyn                      989         7091  2020-03-27
20      Manhattan                      709         4627  2020-03-27
21         Queens                     1810         8529  2020-03-27
22  Staten Island                      321         1534  2020-03-27
23        Unknown                       10           36  2020-03-27
24          Bronx                     1433         5752  2020-03-28
25       Brooklyn                     1372         8129  2020-03-28
26      Manhattan                      916         5237  2020-03-28
27         Queens                     2181         9831  2020-03-28
28  Staten Island                      374         1781  2020-03-28
29        Unknown                       11           35  2020-03-28
30          Bronx                     1729         6250  2020-03-29
31       Brooklyn                     1617         8887  2020-03-29
32      Manhattan                     1074         5582  2020-03-29
33         Queens                     2536        10737  2020-03-29
34  Staten Island                      444         1984  2020-03-29
35        Unknown                       10           34  2020-03-29
36          Bronx                     1880         6925  2020-03-30
37       Brooklyn                     1661        10171  2020-03-30
38      Manhattan                     1075         6060  2020-03-30
39         Queens                     2650        12756  2020-03-30
40  Staten Island                      465         2140  2020-03-30
41        Unknown                       10           35  2020-03-30
42          Bronx                     2056         7815  2020-03-31
43       Brooklyn                     1901        11160  2020-03-31
44      Manhattan                     1130         6538  2020-03-31
45         Queens                     2954        13869  2020-03-31
46  Staten Island                      499         2354  2020-03-31
47        Unknown                        9           35  2020-03-31
48          Bronx                     2414         8608  2020-04-01
49       Brooklyn                     2229        12274  2020-04-01
50      Manhattan                     1287         7021  2020-04-01
51         Queens                     3287        15217  2020-04-01
52  Staten Island                      549         2552  2020-04-01
53        Unknown                        9           35  2020-04-01
54          Bronx                     2657         9344  2020-04-02
55       Brooklyn                     2444        13290  2020-04-02
56      Manhattan                     1337         7397  2020-04-02
57         Queens                     3531        16819  2020-04-0

In [299]:
borough_cases

Borough Total Cases        Date
0            Bronx         106  2020-03-17
1         Brooklyn         236  2020-03-17
2        Manhattan         290  2020-03-17
3           Queens         248  2020-03-17
4    Staten Island          42  2020-03-17
5          Unknown           1  2020-03-17
6            Bronx         303  2020-03-18
7         Brooklyn         505  2020-03-18
8        Manhattan         590  2020-03-18
9           Queens         533  2020-03-18
10   Staten Island          77  2020-03-18
11         Unknown           1  2020-03-18
12           Bronx         496  2020-03-19
13        Brooklyn        1195  2020-03-19
14       Manhattan        1038  2020-03-19
15          Queens        1042  2020-03-19
16   Staten Island         179  2020-03-19
17         Unknown           4  2020-03-19
18           Bronx         736  2020-03-20
19        Brooklyn        1740  2020-03-20
20       Manhattan        1402  2020-03-20
21          Queens        1514  2020-03-20
22   Staten Island         285  2020-03-20
23         Unknown           6  2020-03-20
24           Bronx        1071  2020-03-21
25        Brooklyn        2484  2020-03-21
26       Manhattan        1863  2020-03-21
27          Queens        2254  2020-03-21
28   Staten Island         437  2020-03-21
29         Unknown           6  2020-03-21
30           Bronx        1564  2020-03-22
31        Brooklyn        3154  2020-03-22
32       Manhattan        2324  2020-03-22
33          Queens        3050  2020-03-22
34   Staten Island         666  2020-03-22
35         Unknown           6  2020-03-22
36           Bronx        1999  2020-03-23
37        Brooklyn        3742  2020-03-23
38       Manhattan        2646  2020-03-23
39          Queens        3848  2020-03-23
40   Staten Island         877  2020-03-23
41         Unknown           7  2020-03-23
42           Bronx        2505  2020-03-24
43        Brooklyn        4407  2020-03-24
44       Manhattan        3013  2020-03-24
45          Queens        4667  2020-03-24
46   Staten Island         999  2020-03-24
47         Unknown           6  2020-03-24
48           Bronx        3542  2020-03-25
49        Brooklyn        5232  2020-03-25
50       Manhattan        3616  2020-03-25
51          Queens        6420  2020-03-25
52   Staten Island        1166  2020-03-25
53         Unknown          35  2020-03-25
54           Bronx        4243  2020-03-26
55        Brooklyn        6095  2020-03-26
56       Manhattan        4046  2020-03-26
57          Queens        7362  2020-03-26
58   Staten Island        1330  2020-03-26
59         Unknown          36  2020-03-26
60           Bronx        4880  2020-03-27
61        Brooklyn        7091  2020-03-27
62       Manhattan        4627  2020-03-27
63          Queens        8529  2020-03-27
64   Staten Island        1534  2020-03-27
65         Unknown          36  2020-03-27
66           Bronx        5752  2020-03-28
67        Brooklyn        8129  2020-03-28
68       Manhattan        5237  2020-03-28
69          Queens        9831  2020-03-28
70   Staten Island        1781  2020-03-28
71         Unknown          35  2020-03-28
72           Bronx        6250  2020-03-29
73        Brooklyn        8887  2020-03-29
74       Manhattan        5582  2020-03-29
75          Queens       10737  2020-03-29
76   Staten Island        1984  2020-03-29
77         Unknown          34  2020-03-29
78           Bronx        6925  2020-03-30
79        Brooklyn       10171  2020-03-30
80       Manhattan        6060  2020-03-30
81          Queens       12756  2020-03-30
82   Staten Island        2140  2020-03-30
83         Unknown          35  2020-03-30
84           Bronx        7814  2020-03-31
85        Brooklyn       11160  2020-03-31
86       Manhattan        6539  2020-03-31
87          Queens       13869  2020-03-31
88   Staten Island        2354  2020-03-31
89         Unknown          35  2020-03-31
90           Bronx        8607  2020-04-01
91        Brooklyn       12274  2020-04-01
92       Man

In [304]:
borough_hosp = pd.merge(borough_cases,borough_hosp,how='left',
         on=['Borough','Date']).drop(['Total Cases_y'],axis=1).rename({'Total Cases_x':'Total Cases'},axis=1).fillna(0)

In [308]:
borough_combined = pd.merge(borough_hosp,borough_deaths,how='left',on=['Borough','Date']).fillna(0)
borocols = list(borough_combined.columns)
borocols.remove('Date')
borocols.append('Date')
borough_combined = borough_combined[borocols]
borough_combined

Borough  Total Cases  Ever Hospitalized Cases  \
0            Bronx          106                      0.0   
1         Brooklyn          236                      0.0   
2        Manhattan          290                      0.0   
3           Queens          248                      0.0   
4    Staten Island           42                      0.0   
5          Unknown            1                      0.0   
6            Bronx          303                      0.0   
7         Brooklyn          505                      0.0   
8        Manhattan          590                      0.0   
9           Queens          533                      0.0   
10   Staten Island           77                      0.0   
11         Unknown            1                      0.0   
12           Bronx          496                      0.0   
13        Brooklyn         1195                      0.0   
14       Manhattan         1038                      0.0   
15          Queens         1042                      0.0   
16   Staten Island          179                      0.0   
17         Unknown            4                      0.0   
18           Bronx          736                      0.0   
19        Brooklyn         1740                      0.0   
20       Manhattan         1402                      0.0   
21          Queens         1514                      0.0   
22   Staten Island          285                      0.0   
23         Unknown            6                      0.0   
24           Bronx         1071                      0.0   
25        Brooklyn         2484                      0.0   
26       Manhattan         1863                      0.0   
27          Queens         2254                      0.0   
28   Staten Island          437                      0.0   
29         Unknown            6                      0.0   
30           Bronx         1564                      0.0   
31        Brooklyn         3154                      0.0   
32       Manhattan         2324                      0.0   
33          Queens         3050                      0.0   
34   Staten Island          666                      0.0   
35         Unknown            6                      0.0   
36           Bronx         1999                      0.0   
37        Brooklyn         3742                      0.0   
38       Manhattan         2646                      0.0   
39          Queens         3848                      0.0   
40   Staten Island          877                      0.0   
41         Unknown            7                      0.0   
42           Bronx         2505                    600.0   
43        Brooklyn         4407                    637.0   
44       Manhattan         3013                    417.0   
45          Queens         4667                   1037.0   
46   Staten Island          999                    189.0   
47         Unknown            6                      3.0   
48           Bronx         3542                    802.0   
49        Brooklyn         5232                    919.0   
50       Manhattan         3616                    584.0   
51          Queens         6420                   1363.0   
52   Staten Island         1166                    251.0   
53         Unknown           35                      3.0   
54           Bronx         4243                   1071.0   
55        Brooklyn         6095                    968.0   
56       Manhattan         4046                    729.0   
57          Queens         7362                   1659.0   
58   Staten Island         1330                    283.0   
59         Unknown           36                     10.0   
60           Bronx         4880                   1200.0   
61        Brooklyn         7091                    989.0   
62       Manhattan         4627                    709.0   
63          Queens         8529                   1810.0   
64   Staten Island         1534                    321.0   
65         Unknown           36                    

In [310]:
checktotals(borough_deaths,borough_combined,'Total Deaths')

Sum of first is  17236
Sum of second is  17236.0
Are they equal?


True

#### Output the combined DataFrames to CSV

===================**CSV Output of combined category DataFrames**===============

In [311]:
age_combined.to_csv(folderpath / 'age_combined.csv')
borough_combined.to_csv(folderpath / 'borough_combined.csv')
sex_combined.to_csv(folderpath / 'sex_combined.csv')

#backups to be kept permanent
age_combined.to_csv(folderpath / 'age_combined_orig.csv')
borough_combined.to_csv(folderpath / 'borough_combined_orig.csv')
sex_combined.to_csv(folderpath / 'sex_combined_orig.csv')

========================================================================

In [186]:
# len(newnames[startidx:endidx+1])

In [185]:
# [idx for idx, s in enumerate(newnames) if 'NYCD' in s][0]
# [idx for idx, s in enumerate(newnames) if 'NYCD' in s][-1]

In [184]:
#newnames[startidx+13]

In [183]:
#newnames = [names[i]+dates[i][j]+'.csv' for i in range(len(names)) for j in range(len(dates[i]))]

In [182]:
# newurls = [urls[i]+dates[i][j]+'.pdf' for i in range(len(urls)) for j in range(len(dates[i]))]
# newurls

In [133]:
# newurls = []

# for i in range(len(urls)):
#     current = []
#     for j in range(len(dates[i])):
#         current.append(urls[i]+dates[i][j]+'.pdf')
#         j+=1
#     newurls.extend(current)
#     #i+=1
# #newurls = [urls[0]+dates[0][i]+'.pdf' for i in range(len(dates[0]))]
    
# newurls

In [181]:
#len(dates[0])

In [180]:
#urls[0]+dates[0][0]+'.pdf'

In [51]:

#===============ORIGINAL FILE READ IN======================
# convert PDFs into CSV, read CSVs into DataFrames.

urls = ['https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary-hospitalizations-',
        "https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary-deaths-",
        'https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary-']

names = ['/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc/NYCH.csv',
         '/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc/NYCD.csv',
         '/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc/NYCC.csv']

newurls = [url+pdfdate for url in urls]

for i in range(len(urls)):
    
    tabula.convert_into(urls[i],names[i], output_format="csv", pages='all')
    
nycd = pd.read_csv(folderpath / 'NYCD.csv', header = None)
nych = pd.read_csv(folderpath / 'NYCH.csv', header = None)
nycc = pd.read_csv(folderpath / 'NYCC.csv')

HTTPError: HTTP Error 404: Not Found

In [41]:
nycc

.  Unnamed: 1  Total Cases
0                Total         NaN        45707
1   Median Age (Range)         NaN   49 (0-105)
2            Age Group         NaN          NaN
3           -  0 to 17         NaN     816 (2%)
4          -  18 to 44         NaN  18767 (41%)
5          -  45 to 64         NaN  16104 (35%)
6          -  65 to 74         NaN   5584 (12%)
7       -  75 and over         NaN    4328 (9%)
8           -  Unknown         NaN          108
9      Age 50 and over         NaN          NaN
10              -  Yes         NaN  22061 (48%)
11               -  No         NaN  23538 (52%)
12                 Sex         NaN          NaN
13           -  Female         NaN  20522 (45%)
14             -  Male         NaN  25128 (55%)
15          -  Unknown         NaN           57
16             Borough         NaN          NaN
17            -  Bronx         NaN   8607 (19%)
18         -  Brooklyn         NaN  12274 (27%)
19        -  Manhattan         NaN   7022 (15%)
20           -  Queens         NaN  15217 (33%)
21    -  Staten Island         NaN    2552 (6%)
22          -  Unknown         NaN           35
23              Deaths         NaN         1374

In [42]:
nych

0           1        2
0          Age Group         NaN      NaN
1         -  0 to 17    83 (10%)    816.0
2        -  18 to 44   1711 (9%)  18767.0
3        -  45 to 64  3658 (23%)  16104.0
4        -  65 to 74  2114 (38%)   5584.0
5     -  75 and over  2209 (51%)   4328.0
6         -  Unknown      0 (0%)    108.0
7                Sex         NaN      NaN
8          -  Female  3980 (19%)  20522.0
9            -  Male  5792 (23%)  25128.0
10        -  Unknown      3 (5%)     57.0
11           Borough         NaN      NaN
12          -  Bronx  2414 (28%)   8608.0
13       -  Brooklyn  2229 (18%)  12274.0
14      -  Manhattan  1287 (18%)   7021.0
15         -  Queens  3287 (22%)  15217.0
16  -  Staten Island   549 (22%)   2552.0
17        -  Unknown     9 (26%)     35.0
18             Total  9775 (21%)  45707.0

In [43]:
nycd

0      1     2      3       4
0          Age Group    NaN   NaN    NaN     NaN
1         -  0 to 17    1.0   0.0    0.0     1.0
2        -  18 to 44   51.0   4.0   23.0    78.0
3        -  45 to 64  254.0   9.0   70.0   333.0
4        -  65 to 74  257.0   1.0   76.0   334.0
5     -  75 and over  431.0   4.0  193.0   628.0
6                Sex    NaN   NaN    NaN     NaN
7          -  Female  373.0   4.0  135.0   512.0
8            -  Male  620.0  14.0  227.0   861.0
9         -  Unknown    1.0   0.0    0.0     1.0
10           Borough    NaN   NaN    NaN     NaN
11          -  Bronx  305.0   4.0   51.0   360.0
12       -  Brooklyn  207.0   4.0  117.0   328.0
13      -  Manhattan  113.0   1.0   51.0   165.0
14         -  Queens  311.0   9.0  127.0   447.0
15  -  Staten Island   57.0   0.0   16.0    73.0
16        -  Unknown    1.0   0.0    0.0     1.0
17             Total  994.0  18.0  362.0  1374.0

**Helper functions when reading in the PDFs from NYC.gov**

In [347]:
# testme = test3
# print('test3 is ',test3)
# print('test4 is ',test4)
# try:
#     print(str(testme)[:testme.find('(')-1])
# #rowitem[:rowitem.find('(')-1]
# except:
#     print(int(testme))

test3 is  106
test4 is  106 (32%)
106


In [81]:
#slice the original dataframe and get new dataframes.

def getdfs(df,categories):
    
    for i in range(len(categories)-1):
    
        if df.equals(nych):    
            temp = df[getIndices(df,categories[i])[0][0]:getIndices(df,categories[i+1])[0][0]].rename({0:categories[i], 1:'Ever Hospitalized Cases', 2:'Total Cases'}, axis = 1).reset_index(drop=True)
        elif df.equals(nycd):
            temp = df[getIndices(df,categories[i])[0][0]:getIndices(df,categories[i+1])[0][0]].rename({0:categories[i],1:'Underlying Conditions',
                                                                                                       2:'No Underlying Conditions',3:'Underlying Conditions Pending',
                                                                                                       4:'Total Deaths'}, axis = 1).reset_index(drop=True)
            #temp.drop(['Unnamed: 1'], axis = 1, inplace = True)
        elif df.equals(nycc):
            temp = df[getIndices(df,categories[i])[0][0]:getIndices(df,categories[i+1])[0][0]].rename({'.':categories[i]}, axis = 1).reset_index(drop=True)
            temp.drop(['Unnamed: 1'], axis = 1, inplace = True)
        
        
        temp.drop(0, axis = 0, inplace = True)

        temp[categories[i]] = temp[categories[i]].map(lambda x: x[1:].lstrip())
        temp.iloc[:,1] = temp.iloc[:,1].apply(countform)
        temp['Date'] = today #today

        yield temp

In [799]:
#testdfs = nychdf.copy()
for i in range(len(dates_dict['dateshosp'])):
     

**TODAY'S DATA**

In [103]:
#CREATE DATAFRAMES
age_death,sex_death,borough_death = getdfs(nycd,dcats)
age_hosp,sex_hosp,borough_hosp = getdfs(nych,hcats)
age_cases,cases50new,sex_cases,borough_cases = getdfs(nycc,ccats)

In [105]:
age_death

Age Group  Underlying Conditions  No Underlying Conditions  \
1      0 to 17                      1                       0.0   
2     18 to 44                     51                       4.0   
3     45 to 64                    254                       9.0   
4     65 to 74                    257                       1.0   
5  75 and over                    431                       4.0   

   Underlying Conditions Pending  Total Deaths        Date  
1                            0.0           1.0  2020-04-01  
2                           23.0          78.0  2020-04-01  
3                           70.0         333.0  2020-04-01  
4                           76.0         334.0  2020-04-01  
5                          193.0         628.0  2020-04-01

In [ ]:
age_hosp

In [106]:
#age_hosp = age_hosp.drop(['Total Cases','Date'], axis = 1)
#age_hosp

In [107]:
#age_hosp = pd.merge(age_hosp,age_cases,on = 'Age Group')
#age_hosp

In [108]:
age_cases

Age Group  Total Cases        Date
1      0 to 17          816  2020-04-01
2     18 to 44        18767  2020-04-01
3     45 to 64        16104  2020-04-01
4     65 to 74         5584  2020-04-01
5  75 and over         4328  2020-04-01
6      Unknown            1  2020-04-01

In [109]:
age_combined = mergedfs(age_cases,age_hosp,age_death,'Age Group')
age_combined

Age Group  Ever Hospitalized Cases  Total Cases        Date  \
0      0 to 17                       83          816  2020-04-01   
1     18 to 44                     1711        18767  2020-04-01   
2     45 to 64                     3658        16104  2020-04-01   
3     65 to 74                     2114         5584  2020-04-01   
4  75 and over                     2209         4328  2020-04-01   
5      Unknown                        0            1  2020-04-01   

   Underlying Conditions  No Underlying Conditions  \
0                    1.0                       0.0   
1                   51.0                       4.0   
2                  254.0                       9.0   
3                  257.0                       1.0   
4                  431.0                       4.0   
5                    0.0                       0.0   

   Underlying Conditions Pending  Total Deaths  
0                            0.0           1.0  
1                           23.0          78.0  
2                           70.0         333.0  
3                           76.0         334.0  
4                          193.0         628.0  
5                            0.0           0.0

+++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++

In [110]:
cases50new

Age 50 and over  Total Cases        Date
1             Yes        22061  2020-04-01
2              No        23538  2020-04-01

In [111]:
sex_death

Sex  Underlying Conditions  No Underlying Conditions  \
1   Female                    373                       4.0   
2     Male                    620                      14.0   
3  Unknown                      1                       0.0   

   Underlying Conditions Pending  Total Deaths        Date  
1                          135.0         512.0  2020-04-01  
2                          227.0         861.0  2020-04-01  
3                            0.0           1.0  2020-04-01

In [112]:
sex_hosp

Sex  Ever Hospitalized Cases  Total Cases        Date
1   Female                     3980      20522.0  2020-04-01
2     Male                     5792      25128.0  2020-04-01
3  Unknown                        3         57.0  2020-04-01

In [113]:
sex_cases

Sex  Total Cases        Date
1   Female        20522  2020-04-01
2     Male        25128  2020-04-01
3  Unknown           57  2020-04-01

In [114]:
sex_combined = mergedfs(sex_cases,sex_hosp,sex_death,'Sex')
sex_combined

Sex  Ever Hospitalized Cases  Total Cases        Date  \
0   Female                     3980        20522  2020-04-01   
1     Male                     5792        25128  2020-04-01   
2  Unknown                        3           57  2020-04-01   

   Underlying Conditions  No Underlying Conditions  \
0                    373                       4.0   
1                    620                      14.0   
2                      1                       0.0   

   Underlying Conditions Pending  Total Deaths  
0                          135.0         512.0  
1                          227.0         861.0  
2                            0.0           1.0

+++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++

In [115]:
borough_death

Borough  Underlying Conditions  No Underlying Conditions  \
1          Bronx                    305                       4.0   
2       Brooklyn                    207                       4.0   
3      Manhattan                    113                       1.0   
4         Queens                    311                       9.0   
5  Staten Island                     57                       0.0   
6        Unknown                      1                       0.0   

   Underlying Conditions Pending  Total Deaths        Date  
1                           51.0         360.0  2020-04-01  
2                          117.0         328.0  2020-04-01  
3                           51.0         165.0  2020-04-01  
4                          127.0         447.0  2020-04-01  
5                           16.0          73.0  2020-04-01  
6                            0.0           1.0  2020-04-01

In [116]:
borough_hosp

Borough  Ever Hospitalized Cases  Total Cases        Date
1          Bronx                     2414       8608.0  2020-04-01
2       Brooklyn                     2229      12274.0  2020-04-01
3      Manhattan                     1287       7021.0  2020-04-01
4         Queens                     3287      15217.0  2020-04-01
5  Staten Island                      549       2552.0  2020-04-01
6        Unknown                        9         35.0  2020-04-01

In [117]:
borough_cases

Borough  Total Cases        Date
1          Bronx         8607  2020-04-01
2       Brooklyn        12274  2020-04-01
3      Manhattan         7022  2020-04-01
4         Queens        15217  2020-04-01
5  Staten Island         2552  2020-04-01
6        Unknown           35  2020-04-01

In [118]:
borough_combined = mergedfs(borough_cases,borough_hosp,borough_death,'Borough')
borough_combined

Borough  Ever Hospitalized Cases  Total Cases        Date  \
0          Bronx                     2414         8607  2020-04-01   
1       Brooklyn                     2229        12274  2020-04-01   
2      Manhattan                     1287         7022  2020-04-01   
3         Queens                     3287        15217  2020-04-01   
4  Staten Island                      549         2552  2020-04-01   
5        Unknown                        9           35  2020-04-01   

   Underlying Conditions  No Underlying Conditions  \
0                    305                       4.0   
1                    207                       4.0   
2                    113                       1.0   
3                    311                       9.0   
4                     57                       0.0   
5                      1                       0.0   

   Underlying Conditions Pending  Total Deaths  
0                           51.0         360.0  
1                          117.0         328.0  
2                           51.0         165.0  
3                          127.0         447.0  
4                           16.0          73.0  
5                            0.0           1.0

In [147]:
#underlying_death

#### READ IN PREVIOUS CSVs

**AGE**

In [83]:
age = pd.read_csv(folderpath / 'age.csv')
#age = age.rename({'Count':'Total Deaths'}, axis = 1)
age

Age Group  Ever Hospitalized Cases  Total Cases                 Date  \
0       0 to 17                       67        619.0           2020-03-29   
1      18 to 44                     1459      14233.0           2020-03-29   
2      45 to 64                     2765      11577.0           2020-03-29   
3      65 to 74                     1499       3954.0           2020-03-29   
4   75 and over                     1620       3020.0           2020-03-29   
5       Unknown                        0         71.0           2020-03-29   
6       0 to 17                       72        714.0  2020-03-30 00:00:00   
7      18 to 44                     1448      16028.0  2020-03-30 00:00:00   
8      45 to 64                     2887      13344.0  2020-03-30 00:00:00   
9      65 to 74                     1612       4496.0  2020-03-30 00:00:00   
10  75 and over                     1722       3410.0  2020-03-30 00:00:00   
11      Unknown                        0         95.0  2020-03-30 00:00:00   
12      0 to 17                       72        714.0           2020-03-31   
13     18 to 44                     1448      16028.0           2020-03-31   
14     45 to 64                     2887      13344.0           2020-03-31   
15     65 to 74                     1612       4496.0           2020-03-31   
16  75 and over                     1722       3410.0           2020-03-31   
17      Unknown                        0         95.0           2020-03-31   

    Underlying Conditions  No Underlying Conditions  \
0                     1.0                       0.0   
1                    28.0                       1.0   
2                   140.0                       8.0   
3                   152.0                       1.0   
4                   303.0                       3.0   
5                     0.0                       0.0   
6                     1.0                       0.0   
7                    39.0                       2.0   
8                   170.0                       9.0   
9                   167.0                       0.0   
10                  327.0                       3.0   
11                    0.0                       0.0   
12                    1.0                       0.0   
13                   44.0                       2.0   
14                  180.0                       9.0   
15                  182.0                       0.0   
16                  341.0                       3.0   
17                    0.0                       0.0   

    Underlying Conditions Pending  Total Deaths  
0                             0.0           1.0  
1                            10.0          39.0  
2                            37.0         185.0  
3                            30.0         183.0  
4                            62.0         368.0  
5                             0.0           0.0  
6                             0.0           1.0  
7                            13.0          54.0  
8                            37.0         216.0  
9                            48.0         215.0  
10                           98.0         428.0  
11                            0.0           0.0  
12                            0.0           1.0  
13                           11.0          57.0  
14                           29.0         218.0  
15                           39.0         221.0  
16                           91.0         435.0  
17                            0.0           0.0

In [177]:
# newcols =['Underlying Conditions','No Underlying Conditions','Underlying Conditions Pending']

# for cols in newcols:
#     age[cols] = 0

In [179]:
# rearrange = ['Age Group','Underlying Conditions','No Underlying Conditions','Underlying Conditions Pending',
#              'Total Deaths','Date']

# age = age[rearrange]

In [28]:
# newcols =['Ever Hospitalized Cases','Total Cases','Total Deaths']

# for cols in newcols:
#     age[cols] = 0

In [119]:
age_combined

Age Group  Ever Hospitalized Cases  Total Cases        Date  \
0      0 to 17                       83          816  2020-04-01   
1     18 to 44                     1711        18767  2020-04-01   
2     45 to 64                     3658        16104  2020-04-01   
3     65 to 74                     2114         5584  2020-04-01   
4  75 and over                     2209         4328  2020-04-01   
5      Unknown                        0            1  2020-04-01   

   Underlying Conditions  No Underlying Conditions  \
0                    1.0                       0.0   
1                   51.0                       4.0   
2                  254.0                       9.0   
3                  257.0                       1.0   
4                  431.0                       4.0   
5                    0.0                       0.0   

   Underlying Conditions Pending  Total Deaths  
0                            0.0           1.0  
1                           23.0          78.0  
2                           70.0         333.0  
3                           76.0         334.0  
4                          193.0         628.0  
5                            0.0           0.0

In [120]:
agenew = pd.concat([age,age_combined], axis = 0, sort = False).reset_index(drop = True)
agenew

Age Group  Ever Hospitalized Cases  Total Cases                 Date  \
0       0 to 17                       67        619.0           2020-03-29   
1      18 to 44                     1459      14233.0           2020-03-29   
2      45 to 64                     2765      11577.0           2020-03-29   
3      65 to 74                     1499       3954.0           2020-03-29   
4   75 and over                     1620       3020.0           2020-03-29   
5       Unknown                        0         71.0           2020-03-29   
6       0 to 17                       72        714.0  2020-03-30 00:00:00   
7      18 to 44                     1448      16028.0  2020-03-30 00:00:00   
8      45 to 64                     2887      13344.0  2020-03-30 00:00:00   
9      65 to 74                     1612       4496.0  2020-03-30 00:00:00   
10  75 and over                     1722       3410.0  2020-03-30 00:00:00   
11      Unknown                        0         95.0  2020-03-30 00:00:00   
12      0 to 17                       72        714.0           2020-03-31   
13     18 to 44                     1448      16028.0           2020-03-31   
14     45 to 64                     2887      13344.0           2020-03-31   
15     65 to 74                     1612       4496.0           2020-03-31   
16  75 and over                     1722       3410.0           2020-03-31   
17      Unknown                        0         95.0           2020-03-31   
18      0 to 17                       83        816.0           2020-04-01   
19     18 to 44                     1711      18767.0           2020-04-01   
20     45 to 64                     3658      16104.0           2020-04-01   
21     65 to 74                     2114       5584.0           2020-04-01   
22  75 and over                     2209       4328.0           2020-04-01   
23      Unknown                        0          1.0           2020-04-01   

    Underlying Conditions  No Underlying Conditions  \
0                     1.0                       0.0   
1                    28.0                       1.0   
2                   140.0                       8.0   
3                   152.0                       1.0   
4                   303.0                       3.0   
5                     0.0                       0.0   
6                     1.0                       0.0   
7                    39.0                       2.0   
8                   170.0                       9.0   
9                   167.0                       0.0   
10                  327.0                       3.0   
11                    0.0                       0.0   
12                    1.0                       0.0   
13                   44.0                       2.0   
14                  180.0                       9.0   
15                  182.0                       0.0   
16                  341.0                       3.0   
17                    0.0                       0.0   
18                    1.0                       0.0   
19                   51.0                       4.0   
20                  254.0                       9.0   
21                  257.0                       1.0   
22                  431.0                       4.0   
23                    0.0                       0.0   

    Underlying Conditions Pending  Total Deaths  
0                             0.0           1.0  
1                            10.0          39.0  
2                            37.0         185.0  
3                            30.0         183.0  
4                            62.0         368.0  
5                             0.0           0.0  
6                             0.0           1.0  
7                            13.0          54.0  
8                            37.0         216.0  
9                            48.0         215.0  
10                           98.0         428.0  
11                            0.0           0.0  
12                           

**BOROUGH**

In [122]:
borough = pd.read_csv(folderpath/'borough.csv')
#borough = borough.rename({'Count':'Deaths'}, axis = 1)
borough

Borough  Ever Hospitalized Cases  Total Cases                 Date  \
0           Bronx                     1729       6250.0           2020-03-29   
1        Brooklyn                     1617       8887.0           2020-03-29   
2       Manhattan                     1074       5582.0           2020-03-29   
3          Queens                     2536      10737.0           2020-03-29   
4   Staten Island                      444       1984.0           2020-03-29   
5         Unknown                       10         34.0           2020-03-29   
6           Bronx                     1880       6925.0  2020-03-30 00:00:00   
7        Brooklyn                     1661      10171.0  2020-03-30 00:00:00   
8       Manhattan                     1075       6060.0  2020-03-30 00:00:00   
9          Queens                     2650      12756.0  2020-03-30 00:00:00   
10  Staten Island                      465       2140.0  2020-03-30 00:00:00   
11        Unknown                       10         35.0  2020-03-30 00:00:00   
12          Bronx                     1880       6925.0           2020-03-31   
13       Brooklyn                     1661      10171.0           2020-03-31   
14      Manhattan                     1075       6060.0           2020-03-31   
15         Queens                     2650      12756.0           2020-03-31   
16  Staten Island                      465       2140.0           2020-03-31   
17        Unknown                       10         35.0           2020-03-31   

    Underlying Conditions  No Underlying Conditions  \
0                     176                       0.0   
1                     136                       5.0   
2                      82                       1.0   
3                     194                       7.0   
4                      35                       0.0   
5                       1                       0.0   
6                     198                       1.0   
7                     153                       4.0   
8                      90                       1.0   
9                     222                       8.0   
10                     40                       0.0   
11                      1                       0.0   
12                    210                       1.0   
13                    163                       4.0   
14                     92                       1.0   
15                    238                       8.0   
16                     44                       0.0   
17                      1                       0.0   

    Underlying Conditions Pending  Total Deaths  
0                            12.0         188.0  
1                            44.0         185.0  
2                            20.0         103.0  
3                            52.0         253.0  
4                            11.0          46.0  
5                             0.0           1.0  
6                            16.0         215.0  
7                            59.0         216.0  
8                            28.0         119.0  
9                            75.0         305.0  
10                           18.0          58.0  
11                            0.0           1.0  
12                           19.0         230.0  
13                           48.0         215.0  
14                           26.0         119.0  
15                           63.0         309.0  
16                           14.0          58.0  
17                            0.0           1.0

In [123]:
def assemble_df_new(oldcsv,newdf): #newcols
    #for cols in newcols:
     #   olddf[cols] = 0
    olddf = pd.read_csv(folderpath / oldcsv)
    temp = pd.concat([olddf,newdf], axis = 0, sort = False).reset_index(drop = True)
    #temp = temp.fillna(0)
    return temp

In [124]:
borough_combined

Borough  Ever Hospitalized Cases  Total Cases        Date  \
0          Bronx                     2414         8607  2020-04-01   
1       Brooklyn                     2229        12274  2020-04-01   
2      Manhattan                     1287         7022  2020-04-01   
3         Queens                     3287        15217  2020-04-01   
4  Staten Island                      549         2552  2020-04-01   
5        Unknown                        9           35  2020-04-01   

   Underlying Conditions  No Underlying Conditions  \
0                    305                       4.0   
1                    207                       4.0   
2                    113                       1.0   
3                    311                       9.0   
4                     57                       0.0   
5                      1                       0.0   

   Underlying Conditions Pending  Total Deaths  
0                           51.0         360.0  
1                          117.0         328.0  
2                           51.0         165.0  
3                          127.0         447.0  
4                           16.0          73.0  
5                            0.0           1.0

In [125]:
boroughnew = assemble_df_new('borough.csv',borough_combined)
boroughnew

Borough  Ever Hospitalized Cases  Total Cases                 Date  \
0           Bronx                     1729       6250.0           2020-03-29   
1        Brooklyn                     1617       8887.0           2020-03-29   
2       Manhattan                     1074       5582.0           2020-03-29   
3          Queens                     2536      10737.0           2020-03-29   
4   Staten Island                      444       1984.0           2020-03-29   
5         Unknown                       10         34.0           2020-03-29   
6           Bronx                     1880       6925.0  2020-03-30 00:00:00   
7        Brooklyn                     1661      10171.0  2020-03-30 00:00:00   
8       Manhattan                     1075       6060.0  2020-03-30 00:00:00   
9          Queens                     2650      12756.0  2020-03-30 00:00:00   
10  Staten Island                      465       2140.0  2020-03-30 00:00:00   
11        Unknown                       10         35.0  2020-03-30 00:00:00   
12          Bronx                     1880       6925.0           2020-03-31   
13       Brooklyn                     1661      10171.0           2020-03-31   
14      Manhattan                     1075       6060.0           2020-03-31   
15         Queens                     2650      12756.0           2020-03-31   
16  Staten Island                      465       2140.0           2020-03-31   
17        Unknown                       10         35.0           2020-03-31   
18          Bronx                     2414       8607.0           2020-04-01   
19       Brooklyn                     2229      12274.0           2020-04-01   
20      Manhattan                     1287       7022.0           2020-04-01   
21         Queens                     3287      15217.0           2020-04-01   
22  Staten Island                      549       2552.0           2020-04-01   
23        Unknown                        9         35.0           2020-04-01   

    Underlying Conditions  No Underlying Conditions  \
0                     176                       0.0   
1                     136                       5.0   
2                      82                       1.0   
3                     194                       7.0   
4                      35                       0.0   
5                       1                       0.0   
6                     198                       1.0   
7                     153                       4.0   
8                      90                       1.0   
9                     222                       8.0   
10                     40                       0.0   
11                      1                       0.0   
12                    210                       1.0   
13                    163                       4.0   
14                     92                       1.0   
15                    238                       8.0   
16                     44                       0.0   
17                      1                       0.0   
18                    305                       4.0   
19                    207                       4.0   
20                    113                       1.0   
21                    311                       9.0   
22                     57                       0.0   
23                      1                       0.0   

    Underlying Conditions Pending  Total Deaths  
0                            12.0         188.0  
1                            44.0         185.0  
2                            20.0         103.0  
3                            52.0         253.0  
4                            11.0          46.0  
5                             0.0           1.0  
6                            16.0         215.0  
7                            59.0         216.0  
8                            28.0         119.0  
9                            75.0         305.0  
10                           18.0          58.0  
11                            0.0

**SEX**

In [126]:
sex = pd.read_csv(folderpath/'sex.csv')
# sex = sex.rename({'Count':'Deaths'}, axis = 1)
sex

Sex  Ever Hospitalized Cases  Total Cases                 Date  \
0   Female                     3001      14837.0           2020-03-29   
1     Male                     4408      18593.0           2020-03-29   
2  Unknown                        1         44.0           2020-03-29   
3   Female                     3130      16920.0  2020-03-30 00:00:00   
4     Male                     4610      21120.0  2020-03-30 00:00:00   
5  Unknown                        1         47.0  2020-03-30 00:00:00   
6   Female                     3130      16920.0           2020-03-31   
7     Male                     4610      21120.0           2020-03-31   
8  Unknown                        1         47.0           2020-03-31   

   Underlying Conditions  No Underlying Conditions  \
0                  246.0                       5.0   
1                  378.0                       8.0   
2                    0.0                       0.0   
3                  270.0                       4.0   
4                  433.0                      10.0   
5                    1.0                       0.0   
6                  285.0                       4.0   
7                  462.0                      10.0   
8                    1.0                       0.0   

   Underlying Conditions Pending  Total Deaths  
0                           51.0         302.0  
1                           88.0         474.0  
2                            0.0           0.0  
3                           70.0         344.0  
4                          126.0         569.0  
5                            0.0           1.0  
6                           63.0         352.0  
7                          107.0         579.0  
8                            0.0           1.0

In [127]:
sex_combined

Sex  Ever Hospitalized Cases  Total Cases        Date  \
0   Female                     3980        20522  2020-04-01   
1     Male                     5792        25128  2020-04-01   
2  Unknown                        3           57  2020-04-01   

   Underlying Conditions  No Underlying Conditions  \
0                    373                       4.0   
1                    620                      14.0   
2                      1                       0.0   

   Underlying Conditions Pending  Total Deaths  
0                          135.0         512.0  
1                          227.0         861.0  
2                            0.0           1.0

In [128]:
sexnew = assemble_df_new('sex.csv',sex_combined)
sexnew

Sex  Ever Hospitalized Cases  Total Cases                 Date  \
0    Female                     3001      14837.0           2020-03-29   
1      Male                     4408      18593.0           2020-03-29   
2   Unknown                        1         44.0           2020-03-29   
3    Female                     3130      16920.0  2020-03-30 00:00:00   
4      Male                     4610      21120.0  2020-03-30 00:00:00   
5   Unknown                        1         47.0  2020-03-30 00:00:00   
6    Female                     3130      16920.0           2020-03-31   
7      Male                     4610      21120.0           2020-03-31   
8   Unknown                        1         47.0           2020-03-31   
9    Female                     3980      20522.0           2020-04-01   
10     Male                     5792      25128.0           2020-04-01   
11  Unknown                        3         57.0           2020-04-01   

    Underlying Conditions  No Underlying Conditions  \
0                   246.0                       5.0   
1                   378.0                       8.0   
2                     0.0                       0.0   
3                   270.0                       4.0   
4                   433.0                      10.0   
5                     1.0                       0.0   
6                   285.0                       4.0   
7                   462.0                      10.0   
8                     1.0                       0.0   
9                   373.0                       4.0   
10                  620.0                      14.0   
11                    1.0                       0.0   

    Underlying Conditions Pending  Total Deaths  
0                            51.0         302.0  
1                            88.0         474.0  
2                             0.0           0.0  
3                            70.0         344.0  
4                           126.0         569.0  
5                             0.0           1.0  
6                            63.0         352.0  
7                           107.0         579.0  
8                             0.0           1.0  
9                           135.0         512.0  
10                          227.0         861.0  
11                            0.0           1.0

**CASES50**

In [129]:
cases50current = pd.read_csv(folderpath/'cases50.csv')
cases50current

Age 50 and over  Total Cases                 Date
0             Yes        15706           2020-03-29
1              No        17697           2020-03-29
2             Yes        17958  2020-03-30 00:00:00
3              No        20034  2020-03-30 00:00:00
4             Yes        19481           2020-03-31
5              No        21322           2020-03-31
6             Yes        22061           2020-04-01
7              No        23538           2020-04-01

In [130]:
cases50new

Age 50 and over  Total Cases        Date
1             Yes        22061  2020-04-01
2              No        23538  2020-04-01

In [78]:
cases50current = assemble_df_new('cases50.csv',cases50new)
cases50current

Age 50 and over  Total Cases                 Date
0             Yes        15706           2020-03-29
1              No        17697           2020-03-29
2             Yes        17958  2020-03-30 00:00:00
3              No        20034  2020-03-30 00:00:00
4             Yes        19481           2020-03-31
5              No        21322           2020-03-31

In [110]:
# underlying = pd.read_csv('underlying.csv')
# underlying = underlying.rename({'Count':'Deaths'}, axis = 1)
# underlying

**OUTPUT CSVS**

In [28]:
#only run when starting fresh
# age_combined.to_csv('age.csv', index = False)
# sex_combined.to_csv('sex.csv', index = False)
# borough_combined.to_csv('borough.csv', index = False)
# cases50new.to_csv('cases50.csv', index = False)

In [81]:
agenew.to_csv(folderpath / 'age.csv', index = False)
sexnew.to_csv(folderpath / 'sex.csv', index = False)
boroughnew.to_csv(folderpath / 'borough.csv', index = False)
cases50current.to_csv(folderpath / 'cases50.csv', index = False)

In [82]:
agenew

Age Group  Ever Hospitalized Cases  Total Cases                 Date  \
0       0 to 17                       67        619.0           2020-03-29   
1      18 to 44                     1459      14233.0           2020-03-29   
2      45 to 64                     2765      11577.0           2020-03-29   
3      65 to 74                     1499       3954.0           2020-03-29   
4   75 and over                     1620       3020.0           2020-03-29   
5       Unknown                        0         71.0           2020-03-29   
6       0 to 17                       72        714.0  2020-03-30 00:00:00   
7      18 to 44                     1448      16028.0  2020-03-30 00:00:00   
8      45 to 64                     2887      13344.0  2020-03-30 00:00:00   
9      65 to 74                     1612       4496.0  2020-03-30 00:00:00   
10  75 and over                     1722       3410.0  2020-03-30 00:00:00   
11      Unknown                        0         95.0  2020-03-30 00:00:00   
12      0 to 17                       72        714.0           2020-03-31   
13     18 to 44                     1448      16028.0           2020-03-31   
14     45 to 64                     2887      13344.0           2020-03-31   
15     65 to 74                     1612       4496.0           2020-03-31   
16  75 and over                     1722       3410.0           2020-03-31   
17      Unknown                        0         95.0           2020-03-31   

    Underlying Conditions  No Underlying Conditions  \
0                     1.0                       0.0   
1                    28.0                       1.0   
2                   140.0                       8.0   
3                   152.0                       1.0   
4                   303.0                       3.0   
5                     0.0                       0.0   
6                     1.0                       0.0   
7                    39.0                       2.0   
8                   170.0                       9.0   
9                   167.0                       0.0   
10                  327.0                       3.0   
11                    0.0                       0.0   
12                    1.0                       0.0   
13                   44.0                       2.0   
14                  180.0                       9.0   
15                  182.0                       0.0   
16                  341.0                       3.0   
17                    0.0                       0.0   

    Underlying Conditions Pending  Total Deaths  
0                             0.0           1.0  
1                            10.0          39.0  
2                            37.0         185.0  
3                            30.0         183.0  
4                            62.0         368.0  
5                             0.0           0.0  
6                             0.0           1.0  
7                            13.0          54.0  
8                            37.0         216.0  
9                            48.0         215.0  
10                           98.0         428.0  
11                            0.0           0.0  
12                            0.0           1.0  
13                           11.0          57.0  
14                           29.0         218.0  
15                           39.0         221.0  
16                           91.0         435.0  
17                            0.0           0.0

**=====================+++++++++++++++Corrections++++++++++++++++++++================**
**===================================================================================**

In [2]:
cases50 = pd.read_csv('/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc/cases50.csv')
cases50 #= cases50[0:6]
#cases50.to_csv(folderpath / 'cases50.csv', index = False)

Age 50 and over  Total Cases                 Date
0             Yes        15706           2020-03-29
1              No        17697           2020-03-29
2             Yes        17958  2020-03-30 00:00:00
3              No        20034  2020-03-30 00:00:00
4             Yes        19481           2020-03-31
5              No        21322           2020-03-31
6             Yes        22061           2020-04-01
7              No        23538           2020-04-01
8             Yes        24129           2020-04-02
9              No        25468           2020-04-02

In [37]:
age = pd.read_csv('/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc/age.csv')
age = age[age['Date'] !=  '2020-04-01']#= cases50[0:6]
#age.to_csv(folderpath / 'age.csv', index = False)

In [38]:
sex = pd.read_csv('/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc/sex.csv')
sex = sex[sex['Date'] !=  '2020-04-01']
#sex.to_csv(folderpath / 'sex.csv', index = False)

In [39]:
borough = pd.read_csv('/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc/borough.csv')
borough = borough[borough['Date'] !=  '2020-04-01']
#borough.to_csv(folderpath / 'borough.csv', index = False)

In [ ]:
csvs = ['/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc/age.csv',
        '/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc/sex.csv',
        '/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc/borough.csv',
        '/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc/cases50.csv']

### II. US National Level Data

### III. Global Data

In [3]:
! rm time_series_covid19_deaths_global.csv
! rm time_series_covid19_confirmed_global.csv
! rm time_series_covid19_recovered_global.csv

urls = ['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv']

for url in urls:
    filename = wget.download(url)

In [4]:
confirmed = pd.read_csv('time_series_covid19_confirmed_global.csv')
deaths = pd.read_csv('time_series_covid19_deaths_global.csv')
recovered = pd.read_csv('time_series_covid19_recovered_global.csv') 

In [5]:
confirmed.sample(5)

Province/State         Country/Region        Lat        Long  1/22/20  \
244              NaN  Saint Kitts and Nevis  17.357822  -62.782998        0   
32               NaN             Cabo Verde  16.538800  -23.041800        0   
9    New South Wales              Australia -33.868800  151.209300        0   
110          Mayotte                 France -12.827500   45.166200        0   
146              NaN                 Latvia  56.879600   24.603200        0   

     1/23/20  1/24/20  1/25/20  1/26/20  1/27/20  1/28/20  1/29/20  1/30/20  \
244        0        0        0        0        0        0        0        0   
32         0        0        0        0        0        0        0        0   
9          0        0        0        3        4        4        4        4   
110        0        0        0        0        0        0        0        0   
146        0        0        0        0        0        0        0        0   

     1/31/20  2/1/20  2/2/20  2/3/20  2/4/20  2/5/20  2/6/20  2/7/20  2/8/20  \
244        0       0       0       0       0       0       0       0       0   
32         0       0       0       0       0       0       0       0       0   
9          4       4       4       4       4       4       4       4       4   
110        0       0       0       0       0       0       0       0       0   
146        0       0       0       0       0       0       0       0       0   

     2/9/20  2/10/20  2/11/20  2/12/20  2/13/20  2/14/20  2/15/20  2/16/20  \
244       0        0        0        0        0        0        0        0   
32        0        0        0        0        0        0        0        0   
9         4        4        4        4        4        4        4        4   
110       0        0        0        0        0        0        0        0   
146       0        0        0        0        0        0        0        0   

     2/17/20  2/18/20  2/19/20  2/20/20  2/21/20  2/22/20  2/23/20  2/24/20  \
244        0        0        0        0        0        0        0        0   
32         0        0        0        0        0        0        0        0   
9          4        4        4        4        4        4        4        4   
110        0        0        0        0        0        0        0        0   
146        0        0        0        0        0        0        0        0   

     2/25/20  2/26/20  2/27/20  2/28/20  2/29/20  3/1/20  3/2/20  3/3/20  \
244        0        0        0        0        0       0       0       0   
32         0        0        0        0        0       0       0       0   
9          4        4        4        4        4       6       6      13   
110        0        0        0        0        0       0       0       0   
146        0        0        0        0        0       0       1       1   

     3/4/20  3/5/20  3/6/20  3/7/20  3/8/20  3/9/20  3/10/20  3/11/20  \
244       0       0       0       0       0       0        0        0   
32        0       0       0       0       0       0        0        0   
9        22      22      26      28      38      48       55       65   
110       0       0       0       0       0       0        0        0   
146       1       1       1       1       2       6        8       10   

     3/12/20  3/13/20  3/14/20  3/15/20  3/16/20  3/17/20  3/18/20  3/19/20  \
244        0        0        0        0        0        0        0        0   
32         0        0        0        0        0        0        0        0   
9         65       92      112      134      171      210      267      307   
110        0        0        0        1        1        1        3        3   
146       10       17       26       30       34       49       71       86   

     3/20/20  3/21/20  3/22/20  3/23/20  3/24/20  3/25/20  3/26/20  3/27/20  \
244        0        0        0        0        0        2        2        2   
32         1        3        3        3        3        4        4        5   
9        353      436  

In [ ]:
#confirmed[(confirmed['Province/State'].str.contains('York') == True)|(confirmed['Province/State'].str.contains(', NY') == True)]

In [ ]:
pd.melt?

In [ ]:
np.any?

In [6]:
dates = confirmed.columns[4:]
dates[-5:]
print('The latest data is as of {}.'.format(dates[-1]))

The latest data is as of 3/30/20.


**Convert the dataframes from multiple dates across columns to 1 column containing all the dates. This is for ease of time-series modeling later.**

In [7]:
dates

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20',
       '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20',
       '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20',
       '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20',
       '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20',
       '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20',
       '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20', '3/12/20',
       '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20', '3/18/20',
       '3/19/20', '3/20/20', '3/21/20', '3/22/20', '3/23/20', '3/24/20',
       '3/25/20', '3/26/20', '3/27/20', '3/28/20', '3/29/20', '3/30/20'],
      dtype='object')

In [8]:
confirmed = confirmed.melt(id_vars=['Province/State','Country/Region','Lat','Long'], value_vars=dates, var_name = 'Date',value_name = 'Confirmed')
deaths = deaths.melt(id_vars=['Province/State','Country/Region','Lat','Long'], value_vars=dates, var_name = 'Date',value_name = 'Deaths')
recovered = recovered.melt(id_vars=['Province/State','Country/Region','Lat','Long'], value_vars=dates, var_name = 'Date',value_name = 'Recovered')

In [9]:
#confirmed.head()
#deaths.head()
recovered.head()

Province/State Country/Region      Lat     Long     Date  Recovered
0            NaN    Afghanistan  33.0000  65.0000  1/22/20          0
1            NaN        Albania  41.1533  20.1683  1/22/20          0
2            NaN        Algeria  28.0339   1.6596  1/22/20          0
3            NaN        Andorra  42.5063   1.5218  1/22/20          0
4            NaN         Angola -11.2027  17.8739  1/22/20          0

**Make a combined dataframe**

In [10]:
#combine confirmed,deaths, and recovered into 1 dataframe.

combined = pd.concat([confirmed,deaths['Deaths'],recovered['Recovered']], axis = 1, sort = False)
combined.sample(10)

Province/State                    Country/Region        Lat        Long  \
14086            NaN                            France  46.227600    2.213700   
4996             NaN                       New Zealand -40.900600  174.886000   
3238             NaN  Saint Vincent and the Grenadines  12.984300  -61.287200   
1154             NaN                           Jamaica  18.109600  -77.297500   
6851             NaN                            Kosovo  42.602636   20.902977   
5823             NaN                             Syria  34.802075   38.996815   
5185             NaN                              Fiji -17.713400  178.065000   
9473          Shanxi                             China  37.577700  112.292200   
16149            NaN                           Lebanon  33.854700   35.862300   
979          Bermuda                    United Kingdom  32.307800  -64.750500   

          Date  Confirmed  Deaths  Recovered  
14086  3/17/20       7652     148        2.0  
4996   2/10/20          0       0        0.0  
3238    2/3/20          0       0        0.0  
1154   1/26/20          0       0        0.0  
6851   2/17/20          0       0        0.0  
5823   2/13/20          0       0       60.0  
5185   2/11/20          0       0        0.0  
9473   2/28/20        133       0        0.0  
16149  3/25/20        333       6        1.0  
979    1/25/20          0       0        0.0

In [11]:
#format 'Date' column to date time and then reformat to mm/dd/yy format.
combined['Date'] = pd.to_datetime(combined['Date'], dayfirst = False, yearfirst = False)
combined['Date']= combined['Date'].dt.strftime('%m/%d/%Y')
latest = max(combined['Date'])

In [12]:
combined['Country/Region'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria',
       'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Diamond Princess', 'Cuba', 'Cyprus',
       'Czechia', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador',
       'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon',
       'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala',
       'Guinea', 'Guyana', 'Haiti', 'Holy See', 'Honduras', 'Hungary',
       'Iceland', 'India

In [13]:
#change Korea, South to South Korea as North Korea does not report its cases.

combined.loc[:,'Country/Region'] = combined.loc[:,'Country/Region'].replace('Korea, South','South Korea')

In [ ]:
# #make a US-only dataframe. Can use it for analysis later.

# us = combined.loc[combined['Country/Region'] == 'US']

In [14]:
#back to the global combined. Take out the state level detail because this is global data.
#combined = combined.loc[combined['Province/State'].str.contains(',') != True]
combined.sample(30)

Province/State       Country/Region      Lat      Long  \
12206                   Victoria            Australia -37.8136  144.9631   
1442                         NaN                Niger  17.6078    8.0817   
14485                        NaN              Armenia  40.0691   45.0382   
10434                        NaN           Bangladesh  23.6850   90.3563   
2105                    Shandong                China  36.3427  118.1498   
9289                         NaN           Kyrgyzstan  41.2044   74.7661   
14011                Nova Scotia               Canada  44.6820  -63.7443   
16307                  Chongqing                China  30.0572  107.8740   
3596   Newfoundland and Labrador               Canada  53.1355  -57.6604   
5874                         NaN           Cabo Verde  16.5388  -23.0418   
7794                         NaN      North Macedonia  41.6086   21.7453   
9827                         NaN               Norway  60.4720    8.4689   
9274                         NaN              Iceland  64.9631  -19.0208   
1976                         NaN             Slovenia  46.1512   14.9955   
8973                         NaN  Congo (Brazzaville)  -4.0383   21.7587   
905                          NaN          South Korea  36.0000  128.0000   
15868                        NaN              Germany  51.0000    9.0000   
504       British Virgin Islands       United Kingdom  18.4207  -64.6400   
2296          Northern Territory            Australia -12.4634  130.8456   
6685                    Zhejiang                China  29.1832  120.0934   
670                          NaN           Montenegro  42.5000   19.3000   
7495                         NaN              Hungary  47.1625   19.5033   
13138                        NaN             Portugal  39.3999   -8.2245   
9005                  Martinique               France  14.6415  -61.0242   
7100                         NaN        Guinea-Bissau  11.8037  -15.1804   
4055       Northwest Territories               Canada  64.8255 -124.8457   
17339                      Jilin                China  43.6661  126.1923   
16542                        NaN           Cabo Verde  16.5388  -23.0418   
98                           NaN                Egypt  26.0000   30.0000   
2681                         NaN           Kazakhstan  48.0196   66.9237   

             Date  Confirmed  Deaths  Recovered  
12206  03/10/2020         18       0       20.0  
1442   01/27/2020          0       0        0.0  
14485  03/19/2020        115       0        1.0  
10434  03/03/2020          0       0        0.0  
2105   01/30/2020        158       0        0.0  
9289   02/27/2020          0       0        0.0  
14011  03/17/2020          7       0        0.0  
16307  03/26/2020        578       6       39.0  
3596   02/05/2020          0       0        0.0  
5874   02/14/2020          0       0        0.0  
7794   02/21/2020          0       0        0.0  
9827   02/29/2020         15       0        0.0  
9274   02/27/2020          0       0        0.0  
1976   01/29/2020          0       0        1.0  
8973   02/26/2020          0       0        0.0  
905    01/25/2020          2       0        0.0  
15868  03/24/2020      32986     157        5.0  
504    01/23/2020          0       0        0.0  
2296   01/31/2020          0       0        0.0  
6685   02/17/2020       1171       0        0.0  
670    01/24/2020          0       0        0.0  
7495   02/20/2020          0       0       26.0  
13138  03/13/2020        112       0        0.0  
9005   02/26/2020          0       0        3.0  
7100   02/18/2020          0       0        0.0  
4055   02/06/2020          0       0        0.0  
17339  03/30/2020         98       1        NaN  
16542  03/27/2020          5       1        0.0  
98     01/22/2020          0       0        0.0  
2681   02/01/2020          0       0        9.0

In [15]:
#Fatality Rate and Recovered
combined.loc[:,'Fatality Rate'] = combined['Deaths']/combined['Confirmed']
combined.loc[:,'Fatality Rate'] = combined.loc[:,'Fatality Rate'].fillna(0)
combined.loc[:,'Recovered'] = combined.loc[:,'Recovered'].fillna(0)

In [16]:
#Fill NAs for Province/State so the groupby operation later will work.
combined.loc[:,'Province/State'] = combined.loc[:,'Province/State'].fillna('Unavailable')

In [17]:

combined[(combined['Confirmed']>100) & (combined['Date'] == latest)].sort_values(['Date','Fatality Rate'], ascending = False)[:10]

Province/State  Country/Region      Lat      Long        Date  \
17409    Unavailable           Italy  43.0000   12.0000  03/30/2020   
17463    Unavailable      San Marino  43.9424   12.4578  03/30/2020   
17473    Unavailable           Spain  40.0000   -4.0000  03/30/2020   
17404    Unavailable       Indonesia  -0.7893  113.9213  03/30/2020   
17441    Unavailable     Netherlands  52.1326    5.2913  03/30/2020   
17406    Unavailable            Iraq  33.0000   44.0000  03/30/2020   
17388    Unavailable          France  46.2276    2.2137  03/30/2020   
17405    Unavailable            Iran  32.0000   53.0000  03/30/2020   
17495    Unavailable  United Kingdom  55.3781   -3.4360  03/30/2020   
17370    Unavailable           Egypt  26.0000   30.0000  03/30/2020   

       Confirmed  Deaths  Recovered  Fatality Rate  
17409     101739   11591        0.0       0.113929  
17463        230      25        0.0       0.108696  
17473      87956    7716        0.0       0.087726  
17404       1414     122        0.0       0.086280  
17441      11750     864        0.0       0.073532  
17406        630      46        0.0       0.073016  
17388      44550    3024        0.0       0.067879  
17405      41495    2757        0.0       0.066442  
17495      22141    1408        0.0       0.063592  
17370        656      41        0.0       0.062500

In [99]:
combined[(combined['Country/Region']=='India') & (combined['Date']==latest)]

Province/State Country/Region   Lat  Long        Date  Confirmed  \
15566    Unavailable          India  21.0  78.0  03/25/2020        657   

       Deaths  Fatality Rate  
15566      12       0.018265

In [18]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17526 entries, 0 to 17525
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province/State  17526 non-null  object 
 1   Country/Region  17526 non-null  object 
 2   Lat             17526 non-null  float64
 3   Long            17526 non-null  float64
 4   Date            17526 non-null  object 
 5   Confirmed       17526 non-null  int64  
 6   Deaths          17526 non-null  int64  
 7   Recovered       17526 non-null  float64
 8   Fatality Rate   17526 non-null  float64
dtypes: float64(4), int64(2), object(3)
memory usage: 1.2+ MB


In [ ]:

combined[(combined['Date']=='2/12/20') & (combined['Province/State']=='Hubei')]

In [19]:
#insert a RowID index to track position of items.
combined.insert(0,'RowID',range(1,len(combined.index)+1))

In [20]:
combined.head()

RowID Province/State Country/Region      Lat     Long        Date  \
0      1    Unavailable    Afghanistan  33.0000  65.0000  01/22/2020   
1      2    Unavailable        Albania  41.1533  20.1683  01/22/2020   
2      3    Unavailable        Algeria  28.0339   1.6596  01/22/2020   
3      4    Unavailable        Andorra  42.5063   1.5218  01/22/2020   
4      5    Unavailable         Angola -11.2027  17.8739  01/22/2020   

   Confirmed  Deaths  Recovered  Fatality Rate  
0          0       0        0.0            0.0  
1          0       0        0.0            0.0  
2          0       0        0.0            0.0  
3          0       0        0.0            0.0  
4          0       0        0.0            0.0

In [21]:
#combined.drop('RowID',axis=1,inplace=True)
combined.shape

(17526, 10)

In [22]:
combined.tail()

RowID            Province/State  Country/Region      Lat     Long  \
17521  17522                  Anguilla  United Kingdom  18.2206 -63.0686   
17522  17523    British Virgin Islands  United Kingdom  18.4207 -64.6400   
17523  17524  Turks and Caicos Islands  United Kingdom  21.6940 -71.7979   
17524  17525               Unavailable      MS Zaandam   0.0000   0.0000   
17525  17526               Unavailable        Botswana -22.3285  24.6849   

             Date  Confirmed  Deaths  Recovered  Fatality Rate  
17521  03/30/2020          2       0        0.0            0.0  
17522  03/30/2020          2       0        0.0            0.0  
17523  03/30/2020          5       0        0.0            0.0  
17524  03/30/2020          2       0        0.0            0.0  
17525  03/30/2020          3       0        0.0            0.0

### Create a dataframe grouped by Countries

In [23]:
#does NOT have state/provice granularity.
countries = combined.groupby(['Date','Country/Region'])[['RowID','Date','Country/Region','Confirmed','Deaths','Recovered']].sum().reset_index()
countries

Date      Country/Region  RowID  Confirmed  Deaths  Recovered
0      01/22/2020         Afghanistan      1          0       0        0.0
1      01/22/2020             Albania      2          0       0        0.0
2      01/22/2020             Algeria      3          0       0        0.0
3      01/22/2020             Andorra      4          0       0        0.0
4      01/22/2020              Angola      5          0       0        0.0
...           ...                 ...    ...        ...     ...        ...
12277  03/30/2020           Venezuela  17500        135       3        0.0
12278  03/30/2020             Vietnam  17501        203       0        0.0
12279  03/30/2020  West Bank and Gaza  17514        116       1        0.0
12280  03/30/2020              Zambia  17502         35       0        0.0
12281  03/30/2020            Zimbabwe  17503          7       1        0.0

[12282 rows x 6 columns]

In [24]:
countries.shape

(12282, 6)

In [25]:

print(countries['Date'].unique())
print('\n')
print('latest date is {}'.format(latest))

['01/22/2020' '01/23/2020' '01/24/2020' '01/25/2020' '01/26/2020'
 '01/27/2020' '01/28/2020' '01/29/2020' '01/30/2020' '01/31/2020'
 '02/01/2020' '02/02/2020' '02/03/2020' '02/04/2020' '02/05/2020'
 '02/06/2020' '02/07/2020' '02/08/2020' '02/09/2020' '02/10/2020'
 '02/11/2020' '02/12/2020' '02/13/2020' '02/14/2020' '02/15/2020'
 '02/16/2020' '02/17/2020' '02/18/2020' '02/19/2020' '02/20/2020'
 '02/21/2020' '02/22/2020' '02/23/2020' '02/24/2020' '02/25/2020'
 '02/26/2020' '02/27/2020' '02/28/2020' '02/29/2020' '03/01/2020'
 '03/02/2020' '03/03/2020' '03/04/2020' '03/05/2020' '03/06/2020'
 '03/07/2020' '03/08/2020' '03/09/2020' '03/10/2020' '03/11/2020'
 '03/12/2020' '03/13/2020' '03/14/2020' '03/15/2020' '03/16/2020'
 '03/17/2020' '03/18/2020' '03/19/2020' '03/20/2020' '03/21/2020'
 '03/22/2020' '03/23/2020' '03/24/2020' '03/25/2020' '03/26/2020'
 '03/27/2020' '03/28/2020' '03/29/2020' '03/30/2020']


latest date is 03/30/2020


In [80]:
#countries['Country/Region'].unique()

In [26]:
(countries['Confirmed'].sum() == combined['Confirmed'].sum()) and (countries['Deaths'].sum() == combined['Deaths'].sum())

True

In [27]:
combined[combined['Date'] == latest]['Confirmed'].sum()

782365

In [28]:
countries[countries['Date'] == latest].sort_values(['Confirmed'], ascending = False)

Date                    Country/Region   RowID  Confirmed  \
12270  03/30/2020                                US   17498     161807   
12187  03/30/2020                             Italy   17410     101739   
12255  03/30/2020                             Spain   17474      87956   
12139  03/30/2020                             China  572154      82198   
12168  03/30/2020                           Germany   17393      66885   
12164  03/30/2020                            France  173845      45170   
12183  03/30/2020                              Iran   17406      41495   
12274  03/30/2020                    United Kingdom  175020      22453   
12260  03/30/2020                       Switzerland   17479      15922   
12120  03/30/2020                           Belgium   17296      11899   
12221  03/30/2020                       Netherlands   69762      11817   
12269  03/30/2020                            Turkey   17486      10827   
12254  03/30/2020                       South Korea   17416       9661   
12113  03/30/2020                           Austria   17289       9618   
12135  03/30/2020                            Canada  260495       7398   
12236  03/30/2020                          Portugal   17457       6408   
12186  03/30/2020                            Israel   17409       4695   
12127  03/30/2020                            Brazil   17301       4579   
12227  03/30/2020                            Norway   17448       4445   
12112  03/30/2020                         Australia  138276       4361   
12259  03/30/2020                            Sweden   17478       4028   
12148  03/30/2020                           Czechia   17364       3001   
12185  03/30/2020                           Ireland   17408       2910   
12149  03/30/2020                           Denmark   52098       2755   
12206  03/30/2020                          Malaysia   17426       2626   
12138  03/30/2020                             Chile   17321       2449   
12238  03/30/2020                           Romania   17459       2109   
12235  03/30/2020                            Poland   17456       2055   
12203  03/30/2020                        Luxembourg   17424       1988   
12154  03/30/2020                           Ecuador   17370       1962   
12189  03/30/2020                             Japan   17412       1866   
12239  03/30/2020                            Russia   17460       1836   
12229  03/30/2020                          Pakistan   17450       1717   
12234  03/30/2020                       Philippines   17455       1546   
12264  03/30/2020                          Thailand   17482       1524   
12245  03/30/2020                      Saudi Arabia   17465       1453   
12182  03/30/2020                         Indonesia   17405       1414   
12163  03/30/2020                           Finland   17379       1352   
12253  03/30/2020                      South Africa   17473       1326   
12181  03/30/2020                             India   17404       1251   
12170  03/30/2020                            Greece   17395       1212   
12180  03/30/2020                           Iceland   17403       1086   
12212  03/30/2020                            Mexico   17431        993   
12230  03/30/2020                            Panama   17451        989   
12233  03/30/2020                              Peru   17454        950   
12153  03/30/2020                Dominican Republic   17369        901   
12249  03/30/2020                         Singapore   17469        879   
12110  03/30/2020                         Argentina   17279        820   
12140  03/30/2020                          Colombia   17355        798   
12145  03/30/2020                           Croatia   17360        790   
12247  03/30/2020                            Serbia   17467        785   
12251  03/30/2020                          Slovenia   17471        756   
12159  03/30/2020                           Estonia   17375        715   
12150  03/30/2020

In [29]:
world_graph = countries.groupby(['Date'])[['Confirmed','Deaths','Recovered']].sum().reset_index()
#world_graph = world_graph.melt(id_vars=['Date'], value_vars=['Confirmed','Deaths'], var_name = 'Case',value_name = 'Count')

In [30]:
world_graph['Date']=pd.to_datetime(world_graph['Date'])

In [31]:
world_graph.tail(16)

Date  Confirmed  Deaths  Recovered
53 2020-03-15     167454    6440    83388.0
54 2020-03-16     181574    7126    89486.0
55 2020-03-17     197102    7905   151723.0
56 2020-03-18     214821    8733    95667.0
57 2020-03-19     242500    9867    98750.0
58 2020-03-20     272035   11299    99317.0
59 2020-03-21     304396   12973   116533.0
60 2020-03-22     336953   14651   139254.0
61 2020-03-23     378235   16505   131030.0
62 2020-03-24     418045   18625   136452.0
63 2020-03-25     467653   21181   144930.0
64 2020-03-26     529591   23970   159395.0
65 2020-03-27     593291   27198    25492.0
66 2020-03-28     660706   30652        0.0
67 2020-03-29     720117   33925        0.0
68 2020-03-30     782365   37582        0.0

In [219]:
#world_graph1 = world_graph.pivot(index='Date', columns = 'Case')['Count'].reset_index()

In [32]:
world_graph.loc[:,'New Deaths'] = world_graph.loc[:,'Deaths']
world_graph.loc[0,'New Deaths'] = 0
for i in range(1,len(world_graph['Deaths'])):
    world_graph.loc[i,'New Deaths'] = world_graph.loc[i,'Deaths'] - world_graph.loc[i-1,'Deaths']
    
world_graph.rename({'Deaths':'Total Deaths'}, axis =1, inplace=True)

In [33]:
world_graph['Fatality Rate'] = round((world_graph['Total Deaths']/world_graph['Confirmed'])*100,2)

In [34]:
world_graph.head(10)

Date  Confirmed  Total Deaths  Recovered  New Deaths  Fatality Rate
0 2020-01-22        555            17       28.0           0           3.06
1 2020-01-23        654            18       30.0           1           2.75
2 2020-01-24        941            26       38.0           8           2.76
3 2020-01-25       1434            42       83.0          16           2.93
4 2020-01-26       2118            56       63.0          14           2.64
5 2020-01-27       2927            82      105.0          26           2.80
6 2020-01-28       5578           131      126.0          49           2.35
7 2020-01-29       6166           133      143.0           2           2.16
8 2020-01-30       8234           171      222.0          38           2.08
9 2020-01-31       9927           213      284.0          42           2.15

In [35]:
np.any(world_graph['Fatality Rate'].isnull())

False

**RUN SELECTIVELY**

In [40]:
#create dataframes to prepare for graphing. One with Confirmed vs Total Deaths, and one for Total Deaths vs New Deaths.
world_graph1 = world_graph.melt(id_vars=['Date'], value_vars = ['Confirmed','Total Deaths','New Deaths'], var_name = 'Case', value_name = 'Count')

world_graphCvD = world_graph1[(world_graph1['Case'] == 'Confirmed')|(world_graph1['Case'] == 'Total Deaths')]
# world_graphD = world_graph1[(world_graph1['Case'] == 'New Deaths')|(world_graph1['Case'] == 'Total Deaths')]

===================

In [39]:
#plotly graph to show aggregate confirms vs deaths

# fig = px.area(world_graph1, x = "Date", y = "Count", color= 'Case',
#              title='Cases over time', color_discrete_sequence = ['red', 'green'])
# fig.show()

In [41]:
fig = px.line(world_graphCvD, x="Date", y="Count", color='Case',
             title='Cases over time: Line Plot', color_discrete_sequence = ['cyan', 'red'],template='plotly_dark')



fig.show()

In [42]:
fig = go.Figure()

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(go.Scatter(x=world_graph['Date'], y=world_graph['Total Deaths'],
                    mode='lines',
                    name='Total Deaths',
                    line = dict(color='white', width=4),
                    hovertemplate =
                    '<i>Count</i>: %{y}'+
                    '<br><b>Date</b>: %{x}<br>',                    
                    showlegend = True),
                    secondary_y = False)
fig.add_trace(go.Scatter(x=world_graph['Date'], y=world_graph['New Deaths'],
                    mode='lines',
                    name='New Deaths',
                    line = dict(color='red'),
                    hovertemplate =
                    '<i>Count</i>: %{y}'+
                    '<br><b>Date</b>: %{x}<br>',                    
                    showlegend = True),
                    secondary_y = False)
fig.add_trace(go.Scatter(x=world_graph['Date'], y=world_graph['Fatality Rate'],
                    name='Fatality Rate',
                    line = dict(color='firebrick',dash='dash')),
                    secondary_y = True)

fig.update_layout(template='plotly_dark', title = 'Fatalities Since Jan 22 as of {}'.format(latest))

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="Count", secondary_y=False)
fig.update_yaxes(title_text="% Rate", secondary_y=True)


fig.show()

In [43]:
# fig = px.line(world_graphD, x="Date", y="Count", color='Case',
#              title='Fatalities Since Jan 22', color_discrete_sequence = ['white', 'red'],template='plotly_dark')



# fig.show()